# Mount Google Drive to access files

---
# Imports & Config  

## Config

In [0]:
import os

base_dir = "/content/gdrive/My Drive/Masterarbeit"
object_detection_dir = os.path.join(base_dir, "ObjectDetection")
image_classification = os.path.join(base_dir, "ImageClassification")

classes_dict_file = object_detection_dir + "/classes.pkl"
objects_dict_file = object_detection_dir + "/objects.pkl"
images_dict_file = object_detection_dir + "/images.pkl"

## Check to GPU

In [0]:
import tensorflow as tf
print(tf.__version__)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

1.13.1


SystemError: ignored

## Mount GDrive

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!gcloud config set project objectdetection-229309

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud alpha survey



## Save & Load for pkls

In [0]:
import collections
import pickle as pkl

pkl_vars = {"objects": objects_dict_file,
            "images": images_dict_file,
            "classes": classes_dict_file}

def get_OD(var, force=False):
  if not var in globals() or force:
    print("loading {} from pickle".format(var))
    try:
      with open(pkl_vars[var], "rb") as f:
        return pkl.load(f)
    except:
      return collections.OrderedDict()
  elif var in globals():
    print("using already defined {}".format(var))
    return globals()[var]
  print("Failed to find something existing, creating new OrderedDictioniary for {}".format(var))
  return collections.OrderedDict()


def save_OD(var, local_var = None):
  if local_var is not None:
    print("Saving {} from handover variable".format(var))
    with open(pkl_vars[var], "+wb") as f:
      pkl.dump(local_var, f)
  elif var in globals():
    print("Saving {} from globals".format(var))
    with open(pkl_vars[var], "+wb") as f:
      pkl.dump(globals()[var], f)
  else:
    print("Failed to find something, unable to save {}".format(var))
    
def create_class_id(class_dict, prefix):
  if len(class_dict) == 0:
    class_int = 1
  else:
    class_int = int(next(reversed(class_dict))[2:])+1
  class_id = "{}{}".format(prefix, class_int)
  return class_id, class_int

## Replace print by prettyprint

In [0]:
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=2)
def print(kwargs):
  pp.pprint(kwargs)
  

---
# Object Detection

## Installs

In [0]:
!git clone https://github.com/tensorflow/models.git /content/tf
!cd /content/tf/research; protoc object_detection/protos/*.proto --python_out=.
!cd /content/tf/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python /content/tf/research/object_detection/builders/model_builder_test.py

Cloning into '/content/tf'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 26765 (delta 26), reused 17 (delta 12), pack-reused 26716
Receiving objects: 100% (26765/26765), 508.95 MiB | 33.33 MiB/s, done.
Resolving deltas: 100% (16213/16213), done.
Checking out files: 100% (2983/2983), done.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.102s

OK (skipped=1)


In [0]:
import sys
sys.path.append('tf/research')
sys.path.append('tf/research/slim')

## Imports

In [0]:
import os
import sys

# For running inference on the TF-Hub module.
import tensorflow as tf
import tensorflow_hub as hub

# For downloading the image.
#import matplotlib.pyplot as plt
#import tempfile
#from six.moves.urllib.request import urlopen
#from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import cv2

# For measuring the inference time.
from datetime import datetime

#for xml parsing
from bs4 import BeautifulSoup

W0601 06:23:37.465529 140129679677312 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [0]:
#...
from object_detection.utils import dataset_util, config_util
from object_detection import model_main, exporter


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## Flags

In [0]:
class FLAGS_OD:
  image_dir = os.path.join(object_detection_dir, "Images")
  
  #tf records directory
  tf_records_dir = os.path.join(object_detection_dir, "TFRecords")
  #label_map directory
  label_map_dir = os.path.join(object_detection_dir, "LabelMaps")
  
  #class level to be considered
  class_level = [0,1]
  #how often the class of an object needs to occure to be considered
  min_class_occurence = 20
  
  #dataset split
  testing_percentage = 10
  validation_percentage = 10
  
  #pipeline config
  pipeline_config_dir = os.path.join(object_detection_dir, "ConfigFiles")
  
  #model checkpoints
  model_checkpoint_dir = os.path.join(object_detection_dir, "ModelCheckpoints")
  model_dir = os.path.join(object_detection_dir,"Models")
  
  saved_model_dir = os.path.join(object_detection_dir,"SavedModels")

# Export Model

## copy model

In [0]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Functions to export object detection inference graph."""
import os
import tempfile
import tensorflow as tf
from tensorflow.contrib.quantize.python import graph_matcher
from tensorflow.core.protobuf import saver_pb2
from tensorflow.python.client import session
from tensorflow.python.platform import gfile
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.tools import freeze_graph
from tensorflow.python.training import saver as saver_lib
from object_detection.builders import graph_rewriter_builder
from object_detection.builders import model_builder
from object_detection.core import standard_fields as fields
from object_detection.data_decoders import tf_example_decoder
from object_detection.utils import config_util
from object_detection.utils import shape_utils

slim = tf.contrib.slim

freeze_graph_with_def_protos = freeze_graph.freeze_graph_with_def_protos


def rewrite_nn_resize_op(is_quantized=False):
  """Replaces a custom nearest-neighbor resize op with the Tensorflow version.
  Some graphs use this custom version for TPU-compatibility.
  Args:
    is_quantized: True if the default graph is quantized.
  """
  input_pattern = graph_matcher.OpTypePattern(
      'FakeQuantWithMinMaxVars' if is_quantized else '*')
  reshape_1_pattern = graph_matcher.OpTypePattern(
      'Reshape', inputs=[input_pattern, 'Const'], ordered_inputs=False)
  mul_pattern = graph_matcher.OpTypePattern(
      'Mul', inputs=[reshape_1_pattern, 'Const'], ordered_inputs=False)
  # The quantization script may or may not insert a fake quant op after the
  # Mul. In either case, these min/max vars are not needed once replaced with
  # the TF version of NN resize.
  fake_quant_pattern = graph_matcher.OpTypePattern(
      'FakeQuantWithMinMaxVars',
      inputs=[mul_pattern, 'Identity', 'Identity'],
      ordered_inputs=False)
  reshape_2_pattern = graph_matcher.OpTypePattern(
      'Reshape',
      inputs=[graph_matcher.OneofPattern([fake_quant_pattern, mul_pattern]),
              'Const'],
      ordered_inputs=False)
  add_pattern = graph_matcher.OpTypePattern(
      'Add', inputs=[reshape_2_pattern, '*'], ordered_inputs=False)

  matcher = graph_matcher.GraphMatcher(add_pattern)
  for match in matcher.match_graph(tf.get_default_graph()):
    projection_op = match.get_op(input_pattern)
    reshape_2_op = match.get_op(reshape_2_pattern)
    add_op = match.get_op(add_pattern)
    nn_resize = tf.image.resize_nearest_neighbor(
        projection_op.outputs[0],
        add_op.outputs[0].shape.dims[1:3],
        align_corners=False)

    for index, op_input in enumerate(add_op.inputs):
      if op_input == reshape_2_op.outputs[0]:
        add_op._update_input(index, nn_resize)  # pylint: disable=protected-access
        break


def replace_variable_values_with_moving_averages(graph,
                                                 current_checkpoint_file,
                                                 new_checkpoint_file):
  """Replaces variable values in the checkpoint with their moving averages.
  If the current checkpoint has shadow variables maintaining moving averages of
  the variables defined in the graph, this function generates a new checkpoint
  where the variables contain the values of their moving averages.
  Args:
    graph: a tf.Graph object.
    current_checkpoint_file: a checkpoint containing both original variables and
      their moving averages.
    new_checkpoint_file: file path to write a new checkpoint.
  """
  with graph.as_default():
    variable_averages = tf.train.ExponentialMovingAverage(0.0)
    ema_variables_to_restore = variable_averages.variables_to_restore()
    with tf.Session() as sess:
      read_saver = tf.train.Saver(ema_variables_to_restore)
      read_saver.restore(sess, current_checkpoint_file)
      write_saver = tf.train.Saver()
      write_saver.save(sess, new_checkpoint_file)


def _image_tensor_input_placeholder(input_shape=None):
  """Returns input placeholder and a 4-D uint8 image tensor."""
  if input_shape is None:
    input_shape = (None, None, None, 3)
  input_tensor = tf.placeholder(
      dtype=tf.uint8, shape=input_shape, name='image_tensor')
  return input_tensor, input_tensor


def _tf_example_input_placeholder():
  """Returns input that accepts a batch of strings with tf examples.
  Returns:
    a tuple of input placeholder and the output decoded images.
  """
  batch_tf_example_placeholder = tf.placeholder(
      tf.string, shape=[None], name='tf_example')
  def decode(tf_example_string_tensor):
    tensor_dict = tf_example_decoder.TfExampleDecoder().decode(
        tf_example_string_tensor)
    image_tensor = tensor_dict[fields.InputDataFields.image]
    return image_tensor
  return (batch_tf_example_placeholder,
          shape_utils.static_or_dynamic_map_fn(
              decode,
              elems=batch_tf_example_placeholder,
              dtype=tf.uint8,
              parallel_iterations=32,
              back_prop=False))


def _encoded_image_string_tensor_input_placeholder():
  """Returns input that accepts a batch of PNG or JPEG strings.
  Returns:
    a tuple of input placeholder and the output decoded images.
  """
  batch_image_str_placeholder = tf.placeholder(
      dtype=tf.string,
      shape=[None],
      name='encoded_image_string_tensor')
  def decode(encoded_image_string_tensor):
    image_tensor = tf.image.decode_image(encoded_image_string_tensor,
                                         channels=3)
    image_tensor.set_shape((None, None, 3))
    return image_tensor
  return (batch_image_str_placeholder,
          tf.map_fn(
              decode,
              elems=batch_image_str_placeholder,
              dtype=tf.uint8,
              parallel_iterations=32,
              back_prop=False))


input_placeholder_fn_map = {
    'image_tensor': _image_tensor_input_placeholder,
    'encoded_image_string_tensor':
    _encoded_image_string_tensor_input_placeholder,
    'tf_example': _tf_example_input_placeholder,
}


def add_output_tensor_nodes(postprocessed_tensors,
                            output_collection_name='inference_op'):
  """Adds output nodes for detection boxes and scores.
  Adds the following nodes for output tensors -
    * num_detections: float32 tensor of shape [batch_size].
    * detection_boxes: float32 tensor of shape [batch_size, num_boxes, 4]
      containing detected boxes.
    * detection_scores: float32 tensor of shape [batch_size, num_boxes]
      containing scores for the detected boxes.
    * detection_classes: float32 tensor of shape [batch_size, num_boxes]
      containing class predictions for the detected boxes.
    * detection_keypoints: (Optional) float32 tensor of shape
      [batch_size, num_boxes, num_keypoints, 2] containing keypoints for each
      detection box.
    * detection_masks: (Optional) float32 tensor of shape
      [batch_size, num_boxes, mask_height, mask_width] containing masks for each
      detection box.
  Args:
    postprocessed_tensors: a dictionary containing the following fields
      'detection_boxes': [batch, max_detections, 4]
      'detection_scores': [batch, max_detections]
      'detection_classes': [batch, max_detections]
      'detection_masks': [batch, max_detections, mask_height, mask_width]
        (optional).
      'detection_keypoints': [batch, max_detections, num_keypoints, 2]
        (optional).
      'num_detections': [batch]
    output_collection_name: Name of collection to add output tensors to.
  Returns:
    A tensor dict containing the added output tensor nodes.
  """
  detection_fields = fields.DetectionResultFields
  label_id_offset = 1
  boxes = postprocessed_tensors.get(detection_fields.detection_boxes)
  scores = postprocessed_tensors.get(detection_fields.detection_scores)
  classes = postprocessed_tensors.get(
      detection_fields.detection_classes) + label_id_offset
  keypoints = postprocessed_tensors.get(detection_fields.detection_keypoints)
  masks = postprocessed_tensors.get(detection_fields.detection_masks)
  num_detections = postprocessed_tensors.get(detection_fields.num_detections)
  outputs = {}
  outputs[detection_fields.detection_boxes] = tf.identity(
      boxes, name=detection_fields.detection_boxes)
  outputs[detection_fields.detection_scores] = tf.identity(
      scores, name=detection_fields.detection_scores)
  outputs[detection_fields.detection_classes] = tf.identity(
      classes, name=detection_fields.detection_classes)
  outputs[detection_fields.num_detections] = tf.identity(
      num_detections, name=detection_fields.num_detections)
  if keypoints is not None:
    outputs[detection_fields.detection_keypoints] = tf.identity(
        keypoints, name=detection_fields.detection_keypoints)
  if masks is not None:
    outputs[detection_fields.detection_masks] = tf.identity(
        masks, name=detection_fields.detection_masks)
  for output_key in outputs:
    tf.add_to_collection(output_collection_name, outputs[output_key])

  return outputs


def write_saved_model(saved_model_path,
                      trained_checkpoint_prefix,
                      inputs,
                      outputs):
  """Writes SavedModel to disk.
  If checkpoint_path is not None bakes the weights into the graph thereby
  eliminating the need of checkpoint files during inference. If the model
  was trained with moving averages, setting use_moving_averages to true
  restores the moving averages, otherwise the original set of variables
  is restored.
  Args:
    saved_model_path: Path to write SavedModel.
    frozen_graph_def: tf.GraphDef holding frozen graph.
    inputs: The input placeholder tensor.
    outputs: A tensor dictionary containing the outputs of a DetectionModel.
  """
  saver = tf.train.Saver()
  with session.Session() as sess:
    saver.restore(sess, trained_checkpoint_prefix)

    builder = tf.saved_model.builder.SavedModelBuilder(saved_model_path)

    tensor_info_inputs = {
        'inputs': tf.saved_model.utils.build_tensor_info(inputs)}
    tensor_info_outputs = {}
    for k, v in outputs.items():
      tensor_info_outputs[k] = tf.saved_model.utils.build_tensor_info(v)

    detection_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
            inputs=tensor_info_inputs,
            outputs=tensor_info_outputs,
            method_name=signature_constants.PREDICT_METHOD_NAME))

    builder.add_meta_graph_and_variables(
        sess, [tf.saved_model.tag_constants.SERVING],
        signature_def_map={
            signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
                detection_signature,
        },
    )
    builder.save()


def write_graph_and_checkpoint(inference_graph_def,
                               model_path,
                               input_saver_def,
                               trained_checkpoint_prefix):
  """Writes the graph and the checkpoint into disk."""
  for node in inference_graph_def.node:
    node.device = ''
  with tf.Graph().as_default():
    tf.import_graph_def(inference_graph_def, name='')
    with session.Session() as sess:
      saver = saver_lib.Saver(saver_def=input_saver_def,
                              save_relative_paths=True)
      saver.restore(sess, trained_checkpoint_prefix)
      saver.save(sess, model_path)


def _get_outputs_from_inputs(input_tensors, detection_model,
                             output_collection_name):
  inputs = tf.to_float(input_tensors)
  preprocessed_inputs, true_image_shapes = detection_model.preprocess(inputs)
  output_tensors = detection_model.predict(
      preprocessed_inputs, true_image_shapes)
  postprocessed_tensors = detection_model.postprocess(
      output_tensors, true_image_shapes)
  return add_output_tensor_nodes(postprocessed_tensors,
                                 output_collection_name)


def _build_detection_graph(input_type, detection_model, input_shape,
                           output_collection_name, graph_hook_fn):
  """Build the detection graph."""
  if input_type not in input_placeholder_fn_map:
    raise ValueError('Unknown input type: {}'.format(input_type))
  placeholder_args = {}
  if input_shape is not None:
    if input_type != 'image_tensor':
      raise ValueError('Can only specify input shape for `image_tensor` '
                       'inputs.')
    placeholder_args['input_shape'] = input_shape
  placeholder_tensor, input_tensors = input_placeholder_fn_map[input_type](
      **placeholder_args)
  outputs = _get_outputs_from_inputs(
      input_tensors=input_tensors,
      detection_model=detection_model,
      output_collection_name=output_collection_name)

  # Add global step to the graph.
  slim.get_or_create_global_step()

  if graph_hook_fn: graph_hook_fn()

  return outputs, placeholder_tensor


def _export_inference_graph(input_type,
                            detection_model,
                            use_moving_averages,
                            trained_checkpoint_prefix,
                            output_directory,
                            additional_output_tensor_names=None,
                            input_shape=None,
                            output_collection_name='inference_op',
                            graph_hook_fn=None,
                            write_inference_graph=False):
  """Export helper."""
  tf.gfile.MakeDirs(output_directory)
  frozen_graph_path = os.path.join(output_directory,
                                   'frozen_inference_graph.pb')
  saved_model_path = os.path.join(output_directory, 'saved_model')
  model_path = os.path.join(output_directory, 'model.ckpt')

  outputs, placeholder_tensor = _build_detection_graph(
      input_type=input_type,
      detection_model=detection_model,
      input_shape=input_shape,
      output_collection_name=output_collection_name,
      graph_hook_fn=graph_hook_fn)

  profile_inference_graph(tf.get_default_graph())
  saver_kwargs = {}
  if use_moving_averages:
    # This check is to be compatible with both version of SaverDef.
    if os.path.isfile(trained_checkpoint_prefix):
      saver_kwargs['write_version'] = saver_pb2.SaverDef.V1
      temp_checkpoint_prefix = tempfile.NamedTemporaryFile().name
    else:
      temp_checkpoint_prefix = tempfile.mkdtemp()
    replace_variable_values_with_moving_averages(
        tf.get_default_graph(), trained_checkpoint_prefix,
        temp_checkpoint_prefix)
    checkpoint_to_use = temp_checkpoint_prefix
  else:
    checkpoint_to_use = trained_checkpoint_prefix

  saver = tf.train.Saver(**saver_kwargs)
  input_saver_def = saver.as_saver_def()

  write_graph_and_checkpoint(
      inference_graph_def=tf.get_default_graph().as_graph_def(),
      model_path=model_path,
      input_saver_def=input_saver_def,
      trained_checkpoint_prefix=checkpoint_to_use)
  if write_inference_graph:
    inference_graph_def = tf.get_default_graph().as_graph_def()
    inference_graph_path = os.path.join(output_directory,
                                        'inference_graph.pbtxt')
    for node in inference_graph_def.node:
      node.device = ''
    with gfile.GFile(inference_graph_path, 'wb') as f:
      f.write(str(inference_graph_def))
  
  if additional_output_tensor_names is not None:
    output_node_names = ','.join(outputs.keys()+additional_output_tensor_names)
  else:
    output_node_names = ','.join(outputs.keys())
  
  frozen_graph_def = freeze_graph.freeze_graph_with_def_protos(
      input_graph_def=tf.get_default_graph().as_graph_def(),
      input_saver_def=input_saver_def,
      input_checkpoint=checkpoint_to_use,
      output_node_names=output_node_names,
      restore_op_name='save/restore_all',
      filename_tensor_name='save/Const:0',
      output_graph=frozen_graph_path,
      clear_devices=True,
      initializer_nodes='')

  write_saved_model(saved_model_path, trained_checkpoint_prefix,
                    placeholder_tensor, outputs)


def export_inference_graph(input_type,
                           pipeline_config,
                           trained_checkpoint_prefix,
                           output_directory,
                           input_shape=None,
                           output_collection_name='inference_op',
                           additional_output_tensor_names=None,
                           write_inference_graph=False):
  """Exports inference graph for the model specified in the pipeline config.
  Args:
    input_type: Type of input for the graph. Can be one of ['image_tensor',
      'encoded_image_string_tensor', 'tf_example'].
    pipeline_config: pipeline_pb2.TrainAndEvalPipelineConfig proto.
    trained_checkpoint_prefix: Path to the trained checkpoint file.
    output_directory: Path to write outputs.
    input_shape: Sets a fixed shape for an `image_tensor` input. If not
      specified, will default to [None, None, None, 3].
    output_collection_name: Name of collection to add output tensors to.
      If None, does not add output tensors to a collection.
    additional_output_tensor_names: list of additional output
      tensors to include in the frozen graph.
    write_inference_graph: If true, writes inference graph to disk.
  """
  detection_model = model_builder.build(pipeline_config.model,
                                        is_training=False)
  graph_rewriter_fn = None
  if pipeline_config.HasField('graph_rewriter'):
    graph_rewriter_config = pipeline_config.graph_rewriter
    graph_rewriter_fn = graph_rewriter_builder.build(graph_rewriter_config,
                                                     is_training=False)
  _export_inference_graph(
      input_type,
      detection_model,
      pipeline_config.eval_config.use_moving_averages,
      trained_checkpoint_prefix,
      output_directory,
      additional_output_tensor_names,
      input_shape,
      output_collection_name,
      graph_hook_fn=graph_rewriter_fn,
      write_inference_graph=write_inference_graph)
  pipeline_config.eval_config.use_moving_averages = False
  config_util.save_pipeline_config(pipeline_config, output_directory)


def profile_inference_graph(graph):
  """Profiles the inference graph.
  Prints model parameters and computation FLOPs given an inference graph.
  BatchNorms are excluded from the parameter count due to the fact that
  BatchNorms are usually folded. BatchNorm, Initializer, Regularizer
  and BiasAdd are not considered in FLOP count.
  Args:
    graph: the inference graph.
  """
  tfprof_vars_option = (
      tf.contrib.tfprof.model_analyzer.TRAINABLE_VARS_PARAMS_STAT_OPTIONS)
  tfprof_flops_option = tf.contrib.tfprof.model_analyzer.FLOAT_OPS_OPTIONS

  # Batchnorm is usually folded during inference.
  tfprof_vars_option['trim_name_regexes'] = ['.*BatchNorm.*']
  # Initializer and Regularizer are only used in training.
  tfprof_flops_option['trim_name_regexes'] = [
      '.*BatchNorm.*', '.*Initializer.*', '.*Regularizer.*', '.*BiasAdd.*'
  ]

  tf.contrib.tfprof.model_analyzer.print_model_analysis(
      graph,
      tfprof_options=tfprof_vars_option)

  tf.contrib.tfprof.model_analyzer.print_model_analysis(
      graph,
      tfprof_options=tfprof_flops_option)

In [0]:
!gsutil cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/LabelMaps/2019_5_28_9_24_26.pbtxt labelmap.pbtxt

Copying file:///content/gdrive/My Drive/Masterarbeit/ObjectDetection/LabelMaps/2019_5_28_9_24_26.pbtxt...
/ [1 files][  2.4 KiB/  2.4 KiB]                                                
Operation completed over 1 objects/2.4 KiB.                                      


In [0]:
#!cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/ConfigFiles/faster_rcnn_inception_resnet_v2_atrous_holoselecta.config pipeline.config
#!cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/ConfigFiles/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_holoselecta_sync_colabTPU.config pipeline.config
!cp /content/gdrive/My\ Drive/Masterarbeit/ObjectDetection/ConfigFiles/ssd_mobilenet_v1_075_320x320_holoselecta_sync_colabTPU.config pipeline.config

In [0]:
!rm -r model
!rm -r saved_model
!mkdir model
!gsutil -m cp -r gs://holoselecta_objectdetection/mobilenetv1_075/320/1000/* model/

Copying gs://holoselecta_objectdetection/mobilenetv1_075/320/1000/model.ckpt-0.meta...
Copying gs://holoselecta_objectdetection/mobilenetv1_075/320/1000/checkpoint...
Copying gs://holoselecta_objectdetection/mobilenetv1_075/320/1000/model.ckpt-0.index...
Copying gs://holoselecta_objectdetection/mobilenetv1_075/320/1000/events.out.tfevents.1559314826.27514f5af6fb...
Copying gs://holoselecta_objectdetection/mobilenetv1_075/320/1000/model.ckpt-7000.data-00000-of-00001...
Copying gs://holoselecta_objectdetection/mobilenetv1_075/320/1000/graph.pbtxt...
Copying gs://holoselecta_objectdetection/mobilenetv1_075/320/1000/model.ckpt-7000.index...
Copying gs://holoselecta_objectdetection/mobilenetv1_075/320/1000/model.ckpt-7000.meta...
Copying gs://holoselecta_objectdetection/mobilenetv1_075/320/1000/model.ckpt-0.data-00000-of-00001...
Copying gs://holoselecta_objectdetection/mobilenetv1_075/320/1000/pipeline.config...
Copying gs://holoselecta_objectdetection/mobilenetv1_075/320/1000/eval_validat

In [0]:
import tensorflow as tf

# Assuming object detection API is available for use
from object_detection.utils.config_util import create_pipeline_proto_from_configs
from object_detection.utils.config_util import get_configs_from_pipeline_file

def main(argv):
  
  # Configuration for model to be exported
  config_pathname = "pipeline.config"

  # Input checkpoint for the model to be exported
  # Path to the directory which consists of the saved model on disk
  trained_model_dir = "model"

  # Create proto from model confguration
  configs = get_configs_from_pipeline_file(config_pathname)
  pipeline_proto = create_pipeline_proto_from_configs(configs=configs)

  # Read .ckpt and .meta files from model directory
  checkpoint = tf.train.get_checkpoint_state(trained_model_dir)
  input_checkpoint = checkpoint.model_checkpoint_path
  print(input_checkpoint)

  # Model Version
  model_version_id = "1"

  # Output Directory
  output_directory = "saved_model"

  import shutil
  import os
  try:
    shutil.rmtree(output_directory)
  except:
    pass
  os.mkdir(output_directory)

  # Export model for serving
  export_inference_graph(input_type='encoded_image_string_tensor',pipeline_config=pipeline_proto,trained_checkpoint_prefix=input_checkpoint,output_directory=output_directory)

  

In [0]:
tf.app.run(main)

model/model.ckpt-7000
Instructions for updating:
Colocations handled automatically by placer.


W0531 16:31:43.698338 139991434389376 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/tensor_array_ops.py:162: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


W0531 16:31:43.807965 139991434389376 deprecation.py:323] From <ipython-input-6-fdcc3029d122>:275: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


INFO:tensorflow:depth of additional conv before box predictor: 0


...

I0531 16:31:58.495056 139991434389376 builder_impl.py:414] SavedModel written to: saved_model/saved_model/saved_model.pb


INFO:tensorflow:Writing pipeline config file to saved_model/pipeline.config


I0531 16:31:58.655321 139991434389376 config_util.py:186] Writing pipeline config file to saved_model/pipeline.config


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
!gsutil -m cp -r saved_model/saved_model/* gs://holoselecta_objectdetection/mobilenetv1_075/320/1000/export/1/

Copying file://saved_model/saved_model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://saved_model/saved_model/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://saved_model/saved_model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
-
Operation completed over 3 objects/17.6 MiB.                                     


In [0]:
!mv export/201957194214 export/retinanet_320_1000

In [0]:
import shutil
import os
timestamp = "201957194214"
os.environ["TIMESTAMP_HOLOSELECTA"] = timestamp
!tar -zcvf export/$TIMESTAMP_HOLOSELECTA/holoselecta_encodedimage.tar.gz export/$TIMESTAMP_HOLOSELECTA
try:
  os.mkdir(os.path.join(FLAGS_OD.saved_model_dir, timestamp))
except:
  pass
shutil.copy("export/{}/holoselecta_encodedimage.tar.gz".format(timestamp), os.path.join(FLAGS_OD.saved_model_dir, "{}/holoselecta.tar.gz".format(timestamp)))

export/201957194214/
export/201957194214/variables/
export/201957194214/variables/variables.index
export/201957194214/variables/variables.data-00000-of-00001
export/201957194214/saved_model.pb
tar: export/201957194214: file changed as we read it


'/content/gdrive/My Drive/Masterarbeit/ObjectDetection/SavedModels/201957194214/holoselecta.tar.gz'

## export to gcloud storage

In [0]:
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You must log in to continue. Would you like to log in (Y/n)?  y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&prompt=select_account&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+h

In [0]:
!gsutil cp export/201957194214/holoselecta_encodedimage.tar.gz gs://holoselecta_objectdetection/holoselecta_encodedimage.tar.gz

Copying file://export/201957194214/holoselecta_encodedimage.tar.gz [Content-Type=application/x-tar]...
\
Operation completed over 1 objects/114.3 MiB.                                    


## Evaluate Model

In [0]:
!mkdir export
!gsutil -m cp -r gs://holoselecta_objectdetection/inception/640/1000/export/1/ /content/export/

mkdir: cannot create directory ‘export’: File exists
Copying gs://holoselecta_objectdetection/inception/640/1000/export/1/saved_model.pb...
Copying gs://holoselecta_objectdetection/inception/640/1000/export/1/variables/variables.index...
Copying gs://holoselecta_objectdetection/inception/640/1000/export/1/variables/variables.data-00000-of-00001...
| [3/3 files][249.0 MiB/249.0 MiB] 100% Done                                    
Operation completed over 3 objects/249.0 MiB.                                    


In [0]:
!saved_model_cli show --dir "export/1" --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['inputs'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: encoded_image_string_tensor:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['detection_boxes'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 100, 4)
      name: detection_boxes:0
  outputs['detection_classes'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 100)
      name: detection_classes:0
  outputs['detection_scores'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 100)
      name: detection_scores:0
  outputs['num_detections'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: num_detections:0
Method name is: tensorflow/serving/predict


## Install

In [0]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2343  100  2343    0     0   8192      0 --:--:-- --:--:-- --:--:--  8163
OK
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,609 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:9 http://security.ubu

In [0]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 55 not upgraded.
Need to get 136 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 1.13.0 [136 MB]
Fetched 136 MB in 3s (49.1 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 130791 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_1.13.0_all.deb ...
Unpacking tensorflow-model-server (1.13.0) ...
Setting up tensorflow-model-server (1.13.0) ...


# Test Server

In [0]:
# to kill the model server process
!kill $(pidof tensorflow_model_server)

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


In [0]:
# Copy model


In [0]:
# to start the model server process
%%bash --bg 
nohup tensorflow_model_server \
  --model_name=holoselecta \
  --model_base_path="/content/export" >server.log 2>&1

Starting job # 2 in a separate thread.


In [0]:
# to check the model server is running properly
!tail -50 server.log

2019-04-30 11:13:44.659965: I tensorflow_serving/model_servers/server.cc:82] Building single TensorFlow model file config:  model_name: holoselecta model_base_path: /content/export
2019-04-30 11:13:44.660114: I tensorflow_serving/model_servers/server_core.cc:461] Adding/updating models.
2019-04-30 11:13:44.660136: I tensorflow_serving/model_servers/server_core.cc:558]  (Re-)adding model: holoselecta
2019-04-30 11:13:44.660264: E tensorflow_serving/sources/storage_path/file_system_storage_path_source.cc:369] FileSystemStoragePathSource encountered a file-system access error: Could not find base path /content/export for servable holoselecta
2019-04-30 11:13:45.660413: E tensorflow_serving/sources/storage_path/file_system_storage_path_source.cc:369] FileSystemStoragePathSource encountered a file-system access error: Could not find base path /content/export for servable holoselecta
2019-04-30 11:13:46.660575: E tensorflow_serving/sources/storage_path/file_system_storage_path_source.cc:369]

# Test Request

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
from IPython.display import display
import numpy as np
import json
try:
  import requests
except:
  !pip install -q requests
  import requests
try:
  from tensorflow_serving.apis import predict_pb2
  from tensorflow_serving.apis import prediction_service_pb2_grpc
except:
  !pip install tensorflow-serving-api
  from tensorflow_serving.apis import predict_pb2
  from tensorflow_serving.apis import prediction_service_pb2_grpc
import grpc
from tensorflow.contrib.util import make_tensor_proto
import collections
from time import time
import base64
import io

In [0]:
from object_detection.utils import visualization_utils
from object_detection.utils import label_map_util

## Category Index

In [0]:
def calc_nutriscore(energy, sugar, sat_fat, natrium, protein, fiber, veggieratio, drink, water):
  """calculates the nutriscore for a set of nutritional facts
  Args:
    energy: the energy value in kcal
    sugar: the sugar value in g
    sat_fat: the saturated fats value in g
    salt: the natrium value in mg
    protein: the protein value in g
    fiber: the fiber value in g
    veggieratio: the ratio of veggies, fruits and nuts in [0,1]
  """
  if water:
    return -15, "A"
  
  nutri = {
    "energy": energy,
    "sugar": sugar,
    "sat_fat": sat_fat,
    "natrium": natrium, # salt = 2.54 * natrium (which is equal to sodium)
    "protein": protein,
    "fiber": fiber,
    "veggieratio": veggieratio
  }
  
  if not drink:
    score = 0
    plus = {
      "energy": [0, 80, 160, 240, 320, 400, 480, 560, 640, 720, 800],
      "sugar": [0, 4.5, 9, 13.5, 18, 22.5, 27, 31, 36, 40, 45],
      "sat_fat": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
      "natrium": [0, 0.090, 0.180, 0.270, 0.360, 0.450, 0.540, 0.630, 0.720, 0.810, 0.900]
    }

    minus = {
      "protein": [0, 1.6, 3.2, 4.8, 6.4, 8.0],
      "fiber": [0, 0.9, 1.9, 2.8, 3.7, 4.7],
      "veggieratio": [0, 0.4, 0.6, 0.6, 0.6, 0.8],
    }

    for key,vals in plus.items():
      for it, val in enumerate(reversed(vals)):
        if nutri[key] > val:
          score += (len(vals) - 1 - it)
          break

    for key,vals in minus.items():
      for it, val in enumerate(reversed(vals)):
        if nutri[key] > val:
          score -= (len(vals) - 1 - it)
          break
  
    letters = collections.OrderedDict(
          {"A": -15,
           "B": 0,
           "C": 3,
           "D": 11,
           "E": 19})
  
  else:
    score = 0
    plus = {
      "energy": [0, 1, 30, 60, 90, 120, 150, 180, 210, 240, 270],
      "sugar": [0, 0.1, 1.5, 3, 4.5, 6, 7.5, 9, 10.5, 12, 13.5],
      "sat_fat": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
      "natrium": [0, 90, 180, 270, 360, 450, 540, 630, 720, 810, 900]
    }

    minus = {
      "protein": [0, 1.6, 3.2, 4.8, 6.4, 8.0],
      "fiber": [0, 0.9, 1.9, 2.8, 3.7, 4.7],
      "veggieratio": [0, 0.0, 0.4, 0.4, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.8],
    }

    for key,vals in plus.items():
      for it, val in enumerate(reversed(vals)):
        if nutri[key] > val:
          score += (len(vals) - 1 - it)
          break

    for key,vals in minus.items():
      for it, val in enumerate(reversed(vals)):
        if nutri[key] > val:
          score -= (len(vals) - 1 - it)
          break
    
    letters = collections.OrderedDict(
          {"B": -15,
           "C": 2,
           "D": 6,
           "E": 10})
  
  letter = None
  for key, val in reversed(letters.items()):
    if score >= val:
      letter = key
      break
  
  return score, letter

In [0]:
def per_value_letter(value, typ, drink=False, plus=False):
  """calculates the nutriscore for a set of nutritional facts
  Args:
    energy: the energy value in kcal
    sugar: the sugar value in g
    sat_fat: the saturated fats value in g
    salt: the natrium value in mg
    protein: the protein value in g
    fiber: the fiber value in g
    veggieratio: the ratio of veggies, fruits and nuts in [0,1]
  """
  score = 0
  if not drink:
    if plus:
      vals = {
        "energy": [0.01, 80, 160, 240, 320, 400, 480, 560, 640, 720, 800],
        "sugar": [0.01, 4.5, 9, 13.5, 18, 22.5, 27, 31, 36, 40, 45],
        "sat_fat": [0.01, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        "sodium": [0.01, 0.090, 0.180, 0.270, 0.360, 0.450, 0.540, 0.630, 0.720, 0.810, 0.900]
      }
    else:
      vals = {
        "protein": [0.01, 1.6, 3.2, 4.8, 6.4, 8.0],
        "fiber": [0.01, 0.9, 1.9, 2.8, 3.7, 4.7],
        "veggieratio": [0.01, 0.4, 0.6, 0.6, 0.6, 0.8],
      }
  else:
    if plus:  
      vals = {
        "energy": [0.01, 1, 30, 60, 90, 120, 150, 180, 210, 240, 270],
        "sugar": [0.01, 0.1, 1.5, 3, 4.5, 6, 7.5, 9, 10.5, 12, 13.5],
        "sat_fat": [0.01, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        "sodium": [0.01, 90, 180, 270, 360, 450, 540, 630, 720, 810, 900]
      }
    else:
      vals = {
        "protein": [0, 1.6, 3.2, 4.8, 6.4, 8.0],
        "fiber": [0, 0.9, 1.9, 2.8, 3.7, 4.7],
        "veggieratio": [0, 0.0, 0.4, 0.4, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.8],
      }
    
  for it, val in enumerate(reversed(vals[typ])):
    if value > val:
      score += (len(vals[typ]) - 1 - it)
      break
  
  if plus:
    letters = collections.OrderedDict(
          {"B": 0,
                 "C": 2,
                 "D": 5,
                 "E": 7})
  else:
    letters = collections.OrderedDict(
          {"C": 0,
                 "B": 1,
                 "A": 3})
  
  letter = None
  for key, val in reversed(letters.items()):
    if score >= val:
      letter = key
      break
  
  if letter is None:
    print(score , value, typ, drink, plus)
  
  return letter

In [0]:
def append_nutri_scores(category_index):
  """add the nutriscore to the category index or
  create a new id to nutriscore index
  Args:
    category_index: a dict keyed by the int ids
  Returns:
    
  """
  for key, val in category_index.items():
    if "gtin" == val["name"][0:4]:
      val["gtin"] = val["name"][4:]
    else:
      val["gtin"] = val["name"].split("_")[-1]
    try:
      response = requests.get("https://eatfit-service.foodcoa.ch/products/{}/".format(val["gtin"]),
                             auth=('eatfit_student_tobias', '6swafchC2nvpBwvy'))
      products = response.json()['products']
      val["display_name"] = products[0]["product_name_en"]
      product_nutri = products[0]["nutrients"]
    except:
      continue
    iswater = False
    isdrink = False
    if val["gtin"] in water:
      iswater = True
    elif val["gtin"] in drinks:
      isdrink = True
    for nut in product_nutri:
      natrium = 0
      sat_fat = 0
      sugar = 0
      fiber = 0
      protein = 0
      energy = 0
      if nut["name"] == "sodium":
        natrium = nut["amount"]
        val["sodium"] = natrium
        val["sodiumletter"] = per_value_letter(natrium, "sodium", isdrink, True)
      elif nut["name"] == "saturatedFat":
        sat_fat = nut["amount"]
        val["sat_fat"] = sat_fat
        val["sat_fatletter"] = per_value_letter(sat_fat, "sat_fat", isdrink, True)
      elif nut["name"] == "sugars":
        sugar = nut["amount"]
        val["sugar"] = sugar
        val["sugarletter"] = per_value_letter(sugar, "sugar", isdrink, True)
      elif nut["name"] == "dietaryFiber":
        fiber = nut["amount"]
        val["fibre"] = fiber
        val["fiberletter"] = per_value_letter(fiber, "fiber", isdrink, False)
      elif nut["name"] == "protein":
        protein = nut["amount"]
        val["protein"] = protein
        val["proteinletter"] = per_value_letter(protein, "protein", isdrink, False)
      elif nut["name"] == "energyKcal":
        energy = nut["amount"]
        val["energy"] = energy
        val["energyletter"] = per_value_letter(energy, "energy", isdrink, True)
    val["nutriscore"], val["nutriletter"] = calc_nutriscore(energy, sugar, sat_fat, natrium, protein, fiber, 0, isdrink, iswater)
    
    
    
  return category_index

In [0]:
import shutil
import os
timestamp = "2019_5_28_9_24_26"
shutil.copy(os.path.join(FLAGS_OD.label_map_dir,"{}.pbtxt".format(timestamp)),"label_map.pbtxt")
category_index = label_map_util.create_category_index_from_labelmap("label_map.pbtxt")
category_index = append_nutri_scores(category_index)
print(category_index)

NameError: ignored

### Misc

In [0]:
prods = {
  "7days_croissantschoko_packung_80_1_0000000000003":0,
  "airwaves_menthoneucalyptus_riegel_14_1_50173167":0,
  "balisto_muesli__37g__5000159418546":0,
  "berger_zitronetoertli__50__7610404511007":0,
  "berger_spitzbueb__74__7610404591009":0,
  "bifi_roll__50__4251097402635":0,
  "bueno___43__4008400320328":0,
  "c+swiss_dosenabisicetea__33__9120025930135":1,
  "caferoyal_classicmacchiato__33__7617014164487":0,
  "caprisun_multivitamin__20__4000177605004":1,
  "coke_zero_flasche_50__5449000131836":1,
  "coke__dose_33__54491472":1,
  "coke___50__54491472":1,
  "comella_schokodrink__33__7613100037253":0,
  "darwida_sandwich____7610032065170":0,  
  "evian___50__3068320353500":2,  
  "fanta___50__40822938":1,
  "fini_galaxymix_packung_100_1_8410525150364":0,
  "fini_jellykisses_packung_80_1_8410525116704":0,
  "fuse_peach__50__5449000236623":1,
  "fuse_lemon__50__5449000235947":1,
  "fuse_lemon_dose_33__5449000235947":1,
  "granini_orange__50__4002160092334":1, 
  "henniez_rot__33__7610235000442":2,
  "henniez_gruen__33__7610235000442":2,
  "henniez_blau__33__7610235000442":2,
  "jacklinks_beefjerkyorginal__25__4047751730219":0,
  "jlbrichard_gauffrechoko__90__5410495002079":0,
  "jlbrichard_gauffre__90__5410495002000":0,
  "kagi___50__7610046000259":0,
  "kind_darkchocnutsseasalt__40__0602652176517":0,
  "knoppers_riegel____4035800488808":0,
  "lorenz_nicnacs__40__4018077798818":0,
  "malburner_partysticks__40__7610200279682":0,
  "maltesers___100__5000159023061":0,
  "maltesers_teasers__35__5000159462129":0,
  "milka_tender____0000000000008":0,
  "mnms_gelb__45__40111445":0,
  "mezzomix___50__54490840":1,  
  "oreo___66__84100733":0,
  "pepsi_max__50__4060800105943":1,
  "powerbar_proteinplusschoko__55__4029679520028":0,
  "ragusa___50__76401121":0,
  "ramseier_jusdepomme__50__7610057001078":1,
  "redbull___33__90162909":1,
  "redbull_light__33__90162800":1,
  "rivella_rot__50__7610097111072":1,
  "schweppes_citrus__50__7610238230617":1,
  "stimorol_wildcherry_riegel_14_1_57060330":0,
  "skittles_sour_riegel_51_1_0000000000004":0,
  "snickers___50__5000159461122":0,
  "stimorol_spearmint_riegel_14_1_0000000000005":0,
  "tiki_himbeerbrause_dose_33_1_0000000000001":1,
  "toffifee___33__4014400924275":0,
  "torino___46__76415272":0,
  "twix___50__5000159459228":0,
  "valser_vivabirne__50__7610335001530":1,
  "valser_still__50__7610335002575":2,
  "valser_classic__50__76404160":2,
  "vitaminwell_reload__50__7350042716388":1,
  "volvic_teeminze__50__3057640335648":1,
  "volvic_pinapple__50__305764335648":1,
  "zweifel_graneochilli__100__7610095184009":0,
  "zweifel_paprika__90__7610095013002":0,
}

In [0]:
wrong = {
    "7610404591009" : {
        "name": "berger_spitzbueb__74__7610404591009",
        "gtin":"7610404591009",
        "display_name": "Berger Spitzbuebli",
        "energy":"401.0",
        "sat_fat":"12.3",
        "sugar":"22.5",
        "salt":"0.5",
        "protein":"4.4",
        "fiber":"1.7",
        "healthpercentage": "0",
    },
    "7610200279682" : {
        "name": "malburner_partysticks__40__7610200279682",
        "gtin":"7610200279682",
        "display_name": "Malburner Partysticks",
        "energy":"497.0",
        "sat_fat":"15.0",
        "sugar":"1.0",
        "salt":"5.1",
        "protein":"35.0",
        "fiber":"0.0",
        "healthpercentage": "0",
    },
    "5410495002000" : {
        "name": "jlbrichard_gauffre__90__5410495002000",
        "gtin":"5410495002000",
        "display_name": "J.L. Brichard Gauffre",
        "energy":"447.0",
        "sat_fat":"11.3",
        "sugar":"27.1",
        "salt":"1.1",
        "protein":"6.0",
        "fiber":"1.6",
        "healthpercentage": "0",
    },
    "7617014164487" : {
        "name": "caferoyal_classicmacchiato__33__7617014164487",
        "gtin":"7617014164487",
        "display_name": "Cafe Royal Classic Macchiato",
        "energy":"73.0",
        "sat_fat":"1.5",
        "sugar":"9.7",
        "salt":"0.21",
        "protein":"2.8",
        "fiber":"0.0",
        "healthpercentage": "0",
    },
}
wrong = {}

In [0]:
nutris = {
	"40111445":
	{
		"name":"mnms_gelb__45__40111445",
		"gtin":"40111445",
		"display_name":"M&M's Peanut 45g",
		"energy":"230.0",
		"energyletter":"B",
		"protein":"4.4",
		"proteinletter":"B",
		"sat_fat":"4.6",
		"sat_fatletter":"C",
		"sugar":"24.1",
		"sugarletter":"C",
		"nutriscore":"14",
		"nutriletter":"D",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"76401121":
	{
		"name":"ragusa___50__76401121",
		"gtin":"76401121",
		"display_name":"Ragusa 50g",
		"fiber":"3.0",
		"fiberletter":"A",
		"protein":"7.0",
		"proteinletter":"A",
		"sat_fat":"15.0",
		"sat_fatletter":"E",
		"sugar":"49.0",
		"sugarletter":"E",
		"energy":"576.0",
		"energyletter":"D",
		"nutriscore":"20",
		"nutriletter":"E",
		"fiberletter":"A",
	},
	"76415272":
	{
		"name":"torino___46__76415272",
		"gtin":"76415272",
		"display_name":"Torino Milk 46g",
		"fiber":"2.0",
		"fiberletter":"B",
		"protein":"8.0",
		"proteinletter":"A",
		"sat_fat":"20.0",
		"sat_fatletter":"E",
		"sugar":"47.0",
		"sugarletter":"E",
		"energy":"571.0",
		"energyletter":"D",
		"nutriscore":"21",
		"nutriletter":"E",
		"fiberletter":"B",
	},
	"5000159462129":
	{
		"name":"maltesers_teasers__35__5000159462129",
		"gtin":"5000159462129",
		"display_name":"Maltesers Teasers 35g",
		"sat_fat":"6.3",
		"sat_fatletter":"D",
		"sugar":"18.5",
		"sugarletter":"C",
		"protein":"2.6",
		"proteinletter":"B",
		"energy":"186.0",
		"energyletter":"C",
		"nutriscore":"2",
		"nutriletter":"B",
	},
	"7610046000259":
	{
		"name":"kagi___50__7610046000259",
		"gtin":"7610046000259",
		"display_name":"Kägi fret 25g",
		"energy":"559.0",
		"energyletter":"D",
		"sat_fat":"25.0",
		"sat_fatletter":"E",
		"sugar":"38.0",
		"sugarletter":"E",
		"fiber":"3.1",
		"fiberletter":"A",
		"protein":"6.4",
		"proteinletter":"A",
		"sodium":"0.1",
		"sodiumletter":"B",
		"nutriscore":"19",
		"nutriletter":"E",
	},
	"5000159459228":
	{
		"name":"twix___50__5000159459228",
		"gtin":"5000159459228",
		"display_name":"Twix 50g",
		"energy":"124.0",
		"energyletter":"A",
		"protein":"1.1",
		"proteinletter":"C",
		"sat_fat":"3.5",
		"sat_fatletter":"B",
		"sugar":"12.2",
		"sugarletter":"B",
		"sodium":"0.1732",
		"sodiumletter":"B",
		"nutriscore":"3",
		"nutriletter":"C",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"7610095013002":
	{
		"name":"zweifel_paprika__90__7610095013002",
		"gtin":"7610095013002",
		"display_name":"Zweifel Original Chips Paprika 90g",
		"sodium":"0.5",
		"sodiumletter":"D",
		"sat_fat":"2.0",
		"sat_fatletter":"A",
		"sugar":"5.0",
		"sugarletter":"A",
		"fiber":"5.0",
		"fiberletter":"A",
		"protein":"6.0",
		"proteinletter":"A",
		"energy":"544.0",
		"energyletter":"D",
		"nutriscore":"5",
		"nutriletter":"C",
	},
	"4014400924275":
	{
		"name":"toffifee___33__4014400924275",
		"gtin":"4014400924275",
		"display_name":"Storck Toffifee 33g",
		"energy":"522.0",
		"energyletter":"D",
		"protein":"6.0",
		"proteinletter":"A",
		"sodium":"0.102362205",
		"sodiumletter":"B",
		"sat_fat":"12.7",
		"sat_fatletter":"E",
		"sugar":"48.9",
		"sugarletter":"E",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"5000159418546":
	{
		"name":"balisto_muesli__37g__5000159418546",
		"gtin":"5000159418546",
		"display_name":"Balisto Muesli 37g",
		"energy":"93.0",
		"energyletter":"B",
		"protein":"1.1",
		"proteinletter":"C",
		"sat_fat":"2.0",
		"sat_fatletter":"B",
		"sugar":"8.1",
		"sugarletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"4035800488808":
	{
		"name":"knoppers_riegel____4035800488808",
		"gtin":"4035800488808",
		"display_name":"Knoppers Riegel",
		"energy":"535.0",
		"energyletter":"D",
		"sat_fat":"14.4",
		"sat_fatletter":"E",
		"sugar":"38.8",
		"sugarletter":"E",
		"protein":"8.4",
		"proteinletter":"A",
		"sodium":"0.153543309392603",
		"sodiumletter":"B",
		"nutriscore":"19",
		"nutriletter":"E",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"4008400320328":
	{
		"name":"bueno___43__4008400320328",
		"gtin":"4008400320328",
		"display_name":"Kinder Bueno 43g",
		"protein":"8.6",
		"proteinletter":"A",
		"sugar":"41.2",
		"sugarletter":"E",
		"energy":"122.0",
		"energyletter":"A",
		"sat_fat":"17.3",
		"sat_fatletter":"E",
		"nutriscore":"12",
		"nutriletter":"D",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"7610032065170":
	{
		"name":"darwida_sandwich____7610032065170",
		"gtin":"7610032065170",
		"display_name":"DAR-VIDA Sandwich cream cheese & herbs",
		"sat_fat":"6.0",
		"sat_fatletter":"D",
		"sugar":"3.0",
		"sugarletter":"B",
		"fiber":"5.0",
		"fiberletter":"A",
		"protein":"10.0",
		"proteinletter":"A",
		"sodium":"0.67",
		"sodiumletter":"E",
		"energy":"483.0",
		"energyletter":"D",
		"nutriscore":"6",
		"nutriletter":"C",
	},
	"5000159461122":
	{
		"name":"snickers___50__5000159461122",
		"gtin":"5000159461122",
		"display_name":"Snickers 50g",
		"energy":"241.0",
		"energyletter":"B",
		"protein":"4.3",
		"proteinletter":"B",
		"sat_fat":"4.0",
		"sat_fatletter":"B",
		"sugar":"26.0",
		"sugarletter":"C",
		"nutriscore":"6",
		"nutriletter":"C",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"76404160":
	{
		"name":"valser_classic__50__76404160",
		"gtin":"76404160",
		"display_name":"Valser Sparkling 0.5L PET",
		"sodium":"0.00105",
		"sodiumletter":"B",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"0.0",
		"sugarletter":"A",
		"fiber":"0.0",
		"fiberletter":"C",
		"protein":"0.0",
		"proteinletter":"C",
		"nutriscore":"-15",
		"nutriletter":"A",
		"energyletter":"B",
		"energy":"0.0",
	},
	"3068320353500":
	{
		"name":"evian___50__3068320353500",
		"gtin":"3068320353500",
		"display_name":"Evian Natural 0.5L PET",
		"energy":"0.0",
		"energyletter":"A",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"0.0",
		"sugarletter":"A",
		"sodium":"0.0065",
		"sodiumletter":"B",
		"fiber":"0.0",
		"fiberletter":"C",
		"nutriscore":"-15",
		"nutriletter":"A",
		"fiberletter":"C",
	},
	"305764335648":
	{
		"name":"volvic_pinapple__50__305764335648",
		"gtin":"305764335648",
    "display_name":"Volvic Juicy Pinapple 0,5l",
		"energyletter":"B",
		"nutriscore":"8",
		"nutriletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter": "C",
    "sugar":"6.4",
    "sugarletter": "C",
		"fiber":"0.0",
    "fiberletter": "C",
		"protein":"0.0",
    "proteinletter": "C",
		"energy":"26.0",
      
	},
	"5449000236623":
	{
		"name":"fuse_peach__50__5449000236623",
		"gtin":"5449000236623",
		"display_name":"Fusetea Peach Hibiscus 0.5L PET",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"4.5",
		"sugarletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"nutriscore":"6",
		"nutriletter":"D",
		"energyletter":"B",
		"fiber":"0.0",
    "fiberletter": "C",
		"energy":"19.0",
	},
	"5449000235947":
	{
		"name":"fuse_lemon__50__5449000235947",
		"gtin":"5449000235947",
		"display_name":"Fusetea Lemon Lemongrass 0.5L PET",
		"energy":"19.0",
		"energyletter":"A",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"4.4",
		"sugarletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sodium":"0.012",
		"sodiumletter":"B",
		"nutriscore":"4",
		"nutriletter":"C",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"7610097111072":
	{
		"name":"rivella_rot__50__7610097111072",
		"gtin":"7610097111072",
		"display_name":"Rivella Rot 0,5l",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"fiber":"0.0",
		"fiberletter":"C",
		"protein":"0.0",
		"proteinletter":"C",
		"sugar":"9.0",
		"sugarletter":"D",
		"energy":"37.0",
		"energyletter":"B",
		"nutriscore":"8",
		"nutriletter":"D",
	},
	"90162909":
	{
		"name":"redbull___33__90162909",
		"gtin":"90162909",
		"display_name":"Red Bull : energy drink",
		"energy":"46.0",
		"energyletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"11.0",
		"sugarletter":"E",
		"fiber":"0.0",
		"fiberletter":"C",
		"nutriscore":"10",
		"nutriletter":"E",
	},
	"7610335001530":
	{
		"name":"valser_vivabirne__50__7610335001530",
		"gtin":"7610335001530",
		"display_name":"Valser Viva Pear & Melissa 0.5L PET",
		"energy":"18.0",
		"energyletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sodium":"0.0",
		"sodiumletter":"B",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"4.2",
		"sugarletter":"C",
		"fiber":"0.0",
		"fiberletter":"C",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"7610057001078":
	{
		"name":"ramseier_jusdepomme__50__7610057001078",
		"gtin":"7610057001078",
		"display_name":"Ramseier Suessmost",
		"energy":"113.0",
		"energyletter":"C",
		"protein":"0.5",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"11.0",
		"sugarletter":"E",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"40822938":
	{
		"name":"fanta___50__40822938",
		"gtin":"40822938",
		"display_name":"Fanta Orange 0.33L Glas",
		"energy":"51.0",
		"energyletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sodium":"0.0146",
		"sodiumletter":"B",
		"fiber":"0.0",
		"fiberletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"12.2",
		"sugarletter":"E",
		"nutriscore":"11",
		"nutriletter":"E",
	},
	"54490840":
	{
		"name":"mezzomix___50__54490840",
		"gtin":"54490840",
		"display_name":"Mezzomix 0.5L PET",
		"energy":"43.0",
		"energyletter":"C",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"10.5",
		"sugarletter":"E",
		"sodium":"0.0",
		"sodiumletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"5449000131836":
	{
		"name":"coke_zero_flasche_50__5449000131836",
		"gtin":"5449000131836",
		"display_name":"Coca-Cola zero sugar 0.5L PET",
		"sodium":"0.0",
		"sodiumletter":"B",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"0.0",
		"sugarletter":"A",
		"fiber":"0.0",
		"fiberletter":"C",
		"protein":"0.0",
		"proteinletter":"C",
		"energy":"0.2",
		"energyletter":"A",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"54491472":
	{
		"name":"coke__flasche_50__54491472",
		"gtin":"54491472",
		"display_name":"Coca-Cola 0.5L PET",
		"energy":"42.0",
		"energyletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"10.6",
		"sugarletter":"E",
		"fiber":"0.0",
		"fiberletter":"C",
		"sodium":"0.045",
		"sodiumletter":"B",
		"nutriscore":"10",
		"nutriletter":"E",
	},
	"7622300336738":
	{
		"name":"oreo___154__7622300336738",
		"gtin":"7622300336738",
		"display_name":"Oreo 54g",
		"energy":"480.0",
		"energyletter":"D",
		"sat_fat":"9.8",
		"sat_fatletter":"D",
		"sugar":"38.0",
		"sugarletter":"E",
		"fiber":"0.0",
		"fiberletter":"B",
		"protein":"5.0",
		"proteinletter":"A",
		"sodium":"0.354330713982931",
		"sodiumletter":"C",
		"nutriscore":"20",
		"nutriletter":"E",
	},
	"4251097402635":
	{
		"name":"bifi_roll__50__4251097402635",
		"gtin":"4251097402635",
		"display_name":"Bifi Roll 50g",
		"sugar":"3.5",
		"sugarletter":"A",
		"sat_fat":"14.0",
		"sat_fatletter":"E",
		"protein":"15.0",
		"proteinletter":"A",
		"fiber":"1.5",
		"fiberletter":"B",
		"energy":"450.0",
		"energyletter":"C",
		"nutriscore":"10",
		"nutriletter":"C",
	},
	"9120025930135":
	{
		"name":"c+swiss_dosenabisicetea__33__9120025930135",
		"gtin":"9120025930135",
		"display_name":"C+Swiss Canabis Icetea 0,33l",
		"energy":"31.0",
		"energyletter":"B",
		"sat_fat":"0",
		"sat_fatletter":"B",
		"sugar":"7.4",
		"sugarletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sodium":"0.00787401586628735",
		"sodiumletter":"B",
		"nutriscore":"9",
		"nutriletter":"D",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"4000177605004":
	{
		"name":"caprisun_multivitamin__20__4000177605004",
		"gtin":"4000177605004",
		"display_name":"Capri-Sun - Multi Juice Drink Multivitamin 200ml",
		"sodium":"0.015",
		"sodiumletter":"B",
		"sat_fat":"0.1",
		"sat_fatletter":"A",
		"sugar":"9.0",
		"sugarletter":"D",
		"fiber":"0.0",
		"fiberletter":"C",
		"protein":"0.5",
		"proteinletter":"C",
		"energy":"39.0",
		"energyletter":"B",
		"nutriscore":"8",
		"nutriletter":"D",
	},
	"7613100037253":
	{
		"name":"comella_schokodrink__33__7613100037253",
		"gtin":"7613100037253",
		"display_name":"COMELLA Choco Drink",
		"sat_fat":"0.6",
		"sat_fatletter":"B",
		"protein":"4.5",
		"proteinletter":"B",
		"sugar":"10.0",
		"sugarletter":"C",
		"energy":"66.0",
		"energyletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"7610335002575":
	{
		"name":"valser_still__50__7610335002575",
		"gtin":"7610335002575",
		"display_name":"Valser Still 0.5L PET",
		"energy":"0.0",
		"energyletter":"B",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"0.0",
		"sugarletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"fiber":"0.0",
		"fiberletter":"C",
		"sodium":"3e-05",
		"sodiumletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"90162800":
	{
		"name":"redbull_light__33__90162800",
		"gtin":"90162800",
		"display_name":"Red Bull Sugarfree 25CL",
		"energy":"3.0",
		"energyletter":"A",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"0.0",
		"sugarletter":"A",
		"fiber":"0.0",
		"fiberletter":"C",
		"nutriscore":"1",
		"nutriletter":"B",
	},
	"7610095184009":
	{
		"name":"zweifel_graneochilli__100__7610095184009",
		"gtin":"7610095184009",
		"display_name":"Zweifel Graneo Mild Chilli 100g",
		"fiber":"5.0",
		"fiberletter":"A",
		"energy":"471.0",
		"energyletter":"C",
		"sugar":"7.0",
		"sugarletter":"A",
		"protein":"8.0",
		"proteinletter":"A",
		"sat_fat":"1.5",
		"sat_fatletter":"A",
		"sodium":"0.95",
		"sodiumletter":"E",
		"nutriscore":"8",
		"nutriletter":"C",
	},
	"4047751730219":
	{
		"name":"jacklinks_beefjerkyorginal__25__4047751730219",
		"gtin":"4047751730219",
		"display_name":"Jacklink's Beefjerky Orgininal 25g",
		"energy":"262.0",
		"energyletter":"C",
		"sat_fat":"1.7",
		"sat_fatletter":"B",
		"sugar":"12.0",
		"sugarletter":"C",
		"fiber":"0.0",
		"fiberletter":"C",
		"protein":"42.0",
		"proteinletter":"A",
		"sodium":"1.9",
		"sodiumletter":"E",
		"nutriscore":"4",
		"nutriletter":"C",
	},
	"5000159023061":
	{
		"name":"maltesers___100__5000159023061",
		"gtin":"5000159023061",
		"display_name":"Maltesers Classic 100g",
		"energy":"167.0",
		"energyletter":"B",
		"protein":"2.7",
		"proteinletter":"B",
		"sat_fat":"5.0",
		"sat_fatletter":"C",
		"sugar":"17.2",
		"sugarletter":"B",
		"nutriscore":"13",
		"nutriletter":"D",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"4018077798818":
	{
		"name":"lorenz_nicnacs__40__4018077798818",
		"gtin":"4018077798818",
		"display_name":"Lorenz NicNacs 40g",
		"fiber":"4.4",
		"fiberletter":"A",
		"sat_fat":"11.0",
		"sat_fatletter":"E",
		"sugar":"7.3",
		"sugarletter":"A",
		"energy":"540.0",
		"energyletter":"D",
		"protein":"15.0",
		"proteinletter":"A",
		"nutriscore":"9",
		"nutriletter":"C",
	},
	"7610200279682":
	{
		"name":"malburner_partysticks__40__7610200279682",
		"gtin":"7610200279682",
		"display_name":"Malburner Partysticks 40g",
		"nutriscore":"12",
		"nutriletter":"D",
		"energyletter":"D",
		"sat_fat":"15.0",
    "sat_fatletter":"E",
		"sugar":"1",
    "sugarletter":"B",
		"fiber":"0.0",
    "fiberletter":"C",
		"protein":"36.0",
    "proteinletter":"A",
		"energy":"490.0",
    "natrium": "2",
    "natriumletter":"E"
	},
	"4029679520028":
	{
		"name":"powerbar_proteinplusschoko__55__4029679520028",
		"gtin":"4029679520028",
		"display_name":"PowerBar ProteinPlus 55g",
		"energy":"187.0",
		"energyletter":"C",
		"protein":"17.0",
		"proteinletter":"A",
		"sat_fat":"2.2",
		"sat_fatletter":"C",
		"sugar":"16.0",
		"sugarletter":"C",
		"fiber":"7.4",
		"fiberletter":"A",
		"sodium":"0.1",
		"sodiumletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"50173167":
	{
		"name":"airwaves_menthoneucalyptus_riegel_14_1_50173167",
		"gtin":"50173167",
		"display_name":"Airwaves M&E 14g 2Pck",
		"energy":"142.0",
		"energyletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"0.0",
		"sugarletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"57060330":
	{
		"name":"stimorol_wildcherry_riegel_14_1_57060330",
		"gtin":"57060330",
		"display_name":"Stimorol Wildcherry 14g 2Pck",
		"energy":"182.0",
		"energyletter":"C",
		"protein":"0.4",
		"proteinletter":"C",
		"sat_fat":"0.4",
		"sat_fatletter":"B",
		"sugar":"0.1",
		"sugarletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"8410525150364":
	{
		"name":"fini_galaxymix_packung_100_1_8410525150364",
		"gtin":"8410525150364",
		"display_name":"Fini Galaxy Mix 100g",
		"energy":"320.0",
		"energyletter":"C",
		"protein":"5.0",
		"proteinletter":"A",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"54.0",
		"sugarletter":"E",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"8410525116704":
	{
		"name":"fini_jellykisses_packung_80_1_8410525116704",
		"gtin":"8410525116704",
		"display_name":"Fini Jellykisse 80g",
		"sugar":"58.0",
		"sugarletter":"E",
		"energy":"326.4",
		"energyletter":"C",
		"protein":"4.0",
		"proteinletter":"B",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"5449000235947":
	{
		"name":"fuse_lemon_dose_33__5449000235947",
		"gtin":"5449000235947",
		"display_name":"Fusetea Lemon 0.5L PET",
		"energy":"19.0",
		"energyletter":"A",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"4.4",
		"sugarletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sodium":"0.012",
		"sodiumletter":"B",
		"nutriscore":"4",
		"nutriletter":"C",
		"fiber":"0.0",
    "fiberletter": "C",
	},
	"54491472":
	{
		"name":"coke__dose_33__54491472",
		"gtin":"54491472",
		"display_name":"Coca-Cola 0.5L PET",
		"energy":"42.0",
		"energyletter":"B",
		"protein":"0.0",
		"proteinletter":"C",
		"sat_fat":"0.0",
		"sat_fatletter":"A",
		"sugar":"10.6",
		"sugarletter":"E",
		"fiber":"0.0",
		"fiberletter":"C",
		"sodium":"0.045",
		"sodiumletter":"B",
		"nutriscore":"10",
		"nutriletter":"E",
	},
	"3057640335648":
	{
		"name":"volvic_teeminze__50__3057640335648",
		"gtin":"3057640335648",
		"display_name":"Volvic Thé - Grüntee Minze 0.5L",
		"sodium":"0.004",
		"sodiumletter":"B",
		"sat_fat":"0.0",
		"sat_fatletter":"B",
		"sugar":"4.9",
		"sugarletter":"B",
		"fiber":"0.0",
		"fiberletter":"C",
		"protein":"0.5",
		"proteinletter":"C",
		"energy":"21.0",
		"energyletter":"B",
		"nutriscore":"0",
		"nutriletter":"B",
	},
	"7610238230617":
	{
		"name":"schweppes_citrus__50__7610238230617",
		"gtin":"7610238230617",
		"display_name":"Schweppes Citrus Fusion PET 0,5l",
		"energy":"35",
		"sugar":"8.5",
		"sat_fat":"0",
		"protein":"0",
		"fiber":"0",
    "fiberletter": "C",
		"energyletter":"B",
		"sugarletter":"B",
		"sat_fatletter":"B",
		"proteinletter":"C",
    "nutriscore":"1",
    "nutriletter":"B",
	},
	"0000000000001":
	{
		"name":"tiki_himbeerbrause_dose_33_1_",
		"gtin":"0000000000001",
		"display_name":"Tiki Himbeerbrause 0,25l",
		"energy":"47",
		"sugar":"11",
		"sat_fat":"0",
		"protein":"0",
		"fiber":"0",
    "fiberletter": "C",
		"energyletter":"B",
		"sugarletter":"C",
		"sat_fatletter":"B",
		"proteinletter":"C",
	  "nutriscore":"2",
	  "nutriletter":"B",
	},
	"0000000000002":
	{
		"name":"caferoyal_classicmacchiato__33__",
		"gtin":"0000000000002",
		"display_name":"cafe Royal Classic Macchiato",
		"energy":"20",
		"sugar":"5",
		"sat_fat":"0",
		"protein":"0",
		"fiber":"0",
    "fiberletter": "C",
		"energyletter":"B",
		"sugarletter":"C",
		"sat_fatletter":"B",
		"proteinletter":"C",
		"nutriscore":"5",
		"nutriletter":"C",
	},
	"0000000000003":
	{
		"name":"7days_croissantschoko_packung_80_1_",
		"gtin":"0000000000003",
		"display_name":"7days Croissant Schokolade 80g",
		"energy":"440",
		"sugar":"19",
		"sat_fat":"18",
		"protein":"7",
		"fiber":"3",
    "fiberletter": "B",
		"energyletter":"D",
		"sugarletter":"C",
		"sat_fatletter":"E",
		"proteinletter":"A",
		"nutriscore":"12",
		"nutriletter":"D",
	},
	"0000000000004":
	{
		"name":"skittles_sour_riegel_51_1_",
		"gtin":"0000000000004",
		"display_name":"Skittles Sour 51g",
		"energy":"398",
		"sugar":"87.4",
		"sat_fat":"2.4",
		"protein":"0",
		"fiber":"0",
    "fiberletter": "C",
		"energyletter":"C",
		"sugarletter":"E",
		"sat_fatletter":"C",
		"proteinletter":"C",
		"nutriscore":"16",
		"nutriletter":"D",
	},
	"0000000000005":
	{
		"name":"stimorol_spearmint_riegel_14_1_",
		"gtin":"0000000000005",
		"display_name":"Stimorol Spearmint 14g 2Pck",
		"energy":"160",
		"sugar":"0.1",
		"sat_fat":"0.3",
		"protein":"0.4",
		"fiber":"0",
    "fiberletter": "C",
		"energyletter":"B",
		"sugarletter":"B",
		"sat_fatletter":"B",
		"proteinletter":"C",
		"nutriscore":"1",
		"nutriletter":"B",
	},
	"5410495002079":
	{
		"name":"waffel_schoko____5410495002079",
		"gtin":"5410495002079",
		"display_name":"JL Bernard Gauffre Maxi Liegeoise Cacao",
		"energy":"450",
		"sugar":"28",
		"sat_fat":"15",
		"protein":"6",
		"fiber":"2",
    "fiberletter": "B",
		"energyletter":"D",
		"sugarletter":"D",
		"sat_fatletter":"E",
		"proteinletter":"A",
		"nutriscore":"16",
		"nutriletter":"D",
	},
	"4002160092334":
	{
		"name":"granini_orange__50__4002160092334",
		"gtin":"4002160092334",
		"display_name":"Granini Orangejuice PET 0.2l",
		"energy":"43",
		"sugar":"9",
		"sat_fat":"0",
		"protein":"0",
		"fiber":"0",
    "fiberletter": "B",
		"veggieratio":"0.5",
		"energyletter":"C",
		"sugarletter":"D",
		"sat_fatletter":"B",
		"proteinletter":"C",
		"nutriscore":"5",
		"nutriletter":"C",
	},
	"0000000000008":
	{
		"name":"milka_tender____",
		"gtin":"0000000000008",
		"display_name":"Milka Tender Milchriegel",
		"energy":"425",
		"sugar":"37.5",
		"sat_fat":"12.5",
		"protein":"6",
		"fiber":"0",
    "fiberletter": "C",
		"energyletter":"D",
		"sugarletter":"E",
		"sat_fatletter":"E",
		"proteinletter":"A",
		"nutriscore":"20",
		"nutriletter":"E",
	},
}

In [0]:
import requests
import collections

nutris_new = {}
for key, val in prods.items():
  gtin = key.split("_")[-1]
  nutris_new[gtin] = {"name": key}
  nutris_new[gtin]["gtin"] = gtin
  
    
  if val == 2:
    nutris_new[gtin]["iswater"] = True
  else:
    nutris_new[gtin]["iswater"] = False
  if val == 1:
    nutris_new[gtin]["isdrink"] = True
  else:
    nutris_new[gtin]["isdrink"] = False
  
  if gtin in wrong:
    nutris_new[gtin]["display_name"] = wrong[gtin]["display_name"]
    nutris_new[gtin]["natrium"] = float(wrong[gtin]["natrium"])
    nutris_new[gtin]["sat_fat"] = float(wrong[gtin]["sat_fat"])
    nutris_new[gtin]["sugar"] = float(wrong[gtin]["sugar"])
    nutris_new[gtin]["fiber"] = float(wrong[gtin]["fiber"])
    nutris_new[gtin]["protein"] = float(wrong[gtin]["protein"])
    nutris_new[gtin]["energy"] = float(wrong[gtin]["energy"])
    nutris_new[gtin]["veggieratio"] = float(wrong[gtin]["veggieratio"])
  else:
    try:
      response = requests.get("https://eatfit-service.foodcoa.ch/products/{}/".format(gtin),
                             auth=('eatfit_student_tobias', '6swafchC2nvpBwvy'))
      products = response.json()['products']
      nutris_new[gtin]["display_name"] = products[0]["product_name_en"]
      product_nutri = products[0]["nutrients"]
      
      if "health_percentage" in products[0]:
        try:
          nutris_new[gtin]["veggieratio"] = products[0]["health_percentage"] / 100.0
        except:
          nutris_new[gtin]["veggieratio"] = 0
      
    except:
      if gtin in nutris:
        nutris_new[gtin]["display_name"] = nutris[gtin]["display_name"]
        nutris_new[gtin]["natrium"] = 0
        nutris_new[gtin]["sat_fat"] = float(nutris[gtin]["sat_fat"])
        nutris_new[gtin]["sugar"] = float(nutris[gtin]["sugar"])
        nutris_new[gtin]["fiber"] = float(nutris[gtin]["fiber"])
        nutris_new[gtin]["protein"] = float(nutris[gtin]["protein"])
        nutris_new[gtin]["energy"] = float(nutris[gtin]["energy"])
        nutris_new[gtin]["veggieratio"] = 0
      else:
        print(key)

    found_salt = False
    for nut in product_nutri:
      if nut["name"] == "sodium" and not found_salt:
        nutris_new[gtin]["natrium"] = nut["amount"]

      elif nut["name"] == "salt":
        nutris_new[gtin]["natrium"] = nut["amount"] / 2.54

        found_salt = True
      elif nut["name"] == "saturatedFat":
        nutris_new[gtin]["sat_fat"] = nut["amount"]

      elif nut["name"] == "sugars":
        nutris_new[gtin]["sugar"] = nut["amount"]

      elif nut["name"] == "dietaryFiber":
        nutris_new[gtin]["fiber"] = nut["amount"]

      elif nut["name"] == "protein":
        nutris_new[gtin]["protein"] = nut["amount"]

      elif nut["name"] == "energyKcal":
        nutris_new[gtin]["energy"] = nut["amount"]


    
    
    
  if not "natrium" in nutris_new[gtin]:
    nutris_new[gtin]["natrium"] = 0
  if not "sat_fat" in nutris_new[gtin]:
    if gtin in nutris and "sat_fat" in nutris[gtin]:
      nutris_new[gtin]["sat_fat"] = float(nutris[gtin]["sat_fat"])
    else:
      nutris_new[gtin]["sat_fat"] = 0
  if not "sugar" in nutris_new[gtin]:
    if gtin in nutris and "sugar" in nutris[gtin]:
      nutris_new[gtin]["sugar"] = float(nutris[gtin]["sugar"])
    else:
      nutris_new[gtin]["sugar"] = 0
  if not "fiber" in nutris_new[gtin]:
    if gtin in nutris and "fiber" in nutris[gtin]:
      nutris_new[gtin]["fiber"] = float(nutris[gtin]["fiber"])
    else:
      nutris_new[gtin]["fiber"] = 0
  if not "protein" in nutris_new[gtin]:
    if gtin in nutris and "protein" in nutris[gtin]:
      nutris_new[gtin]["protein"] = float(nutris[gtin]["protein"])
    else:
      nutris_new[gtin]["protein"] = 0
  if not "energy" in nutris_new[gtin]:
    if gtin in nutris and "energy" in nutris[gtin]:
      nutris_new[gtin]["energy"] = float(nutris[gtin]["energy"])
    else:
      nutris_new[gtin]["energy"] = 0
  if not "veggieratio" in nutris_new[gtin]:
    nutris_new[gtin]["veggieratio"] = 0
      
      
  nutris_new[gtin]["natriumletter"] = per_value_letter(nutris_new[gtin]["natrium"], "sodium", nutris_new[gtin]["isdrink"], True)
  nutris_new[gtin]["sat_fatletter"] = per_value_letter(nutris_new[gtin]["sat_fat"], "sat_fat", nutris_new[gtin]["isdrink"], True)
  nutris_new[gtin]["sugarletter"] = per_value_letter(nutris_new[gtin]["sugar"], "sugar", nutris_new[gtin]["isdrink"], True)
  nutris_new[gtin]["fiberletter"] = per_value_letter(nutris_new[gtin]["fiber"], "fiber", nutris_new[gtin]["isdrink"], False)
  nutris_new[gtin]["proteinletter"] = per_value_letter(nutris_new[gtin]["protein"], "protein", nutris_new[gtin]["isdrink"], False)
  nutris_new[gtin]["energyletter"] = per_value_letter(nutris_new[gtin]["energy"], "energy", nutris_new[gtin]["isdrink"], True)
  nutris_new[gtin]["veggieratioletter"] = per_value_letter(nutris_new[gtin]["veggieratio"], "veggieratio", nutris_new[gtin]["isdrink"], False)
  
  
  nutris_new[gtin]["nutriscore"], nutris_new[gtin]["nutriletter"] = calc_nutriscore(nutris_new[gtin]["energy"],
                                                          nutris_new[gtin]["sugar"],
                                                          nutris_new[gtin]["sat_fat"],
                                                          nutris_new[gtin]["natrium"],
                                                          nutris_new[gtin]["protein"],
                                                          nutris_new[gtin]["fiber"],
                                                          nutris_new[gtin]["veggieratio"],
                                                          nutris_new[gtin]["isdrink"],
                                                          nutris_new[gtin]["iswater"])

In [0]:
for key, val in category_index.items():
  if val["gtin"] in nutris:
    print('\t"{}":'.format(val["gtin"]))
    print("\t{")
    ks = []
    for k in val.keys():
      if k not in ks:
        ks.append(k)
    for k in nutris[val["gtin"]].keys():
      if k not in ks:
        ks.append(k)
    if "energy" in k and not "energyletter" in k:
      if k in nutris[val["gtin"]]:
        val["energyletter"] = per_value_letter(float(nutris[val["gtin"]][k]), "energy", drink=(val["gtin"] in drinks), plus=True)
      else:
        val["energyletter"] = per_value_letter(float(val[k]), "energy", drink=(val["gtin"] in drinks), plus=True)
    if "sugar" in k and not "sugarletter" in k:
      if k in nutris[val["gtin"]]:
        val["sugarletter"] = per_value_letter(float(nutris[val["gtin"]][k]), "sugar", drink=(val["gtin"] in drinks), plus=True)
      else:
        val["sugarletter"] = per_value_letter(float(val[k]), "sugar", drink=(val["gtin"] in drinks), plus=True)
    if "sat_fat" in k and not "sat_fatletter" in k:
      if k in nutris[val["gtin"]]:
        val["sat_fatletter"] = per_value_letter(float(nutris[val["gtin"]][k]), "sat_fat", drink=(val["gtin"] in drinks), plus=True)
      else:
        val["sat_fatletter"] = per_value_letter(float(val[k]), "sat_fat", drink=(val["gtin"] in drinks), plus=True)
    if "sodium" in k and not "sodiumletter" in k:
      if k in nutris[val["gtin"]]:
        val["sodiumletter"] = per_value_letter(float(nutris[val["gtin"]][k]), "sodium", drink=(val["gtin"] in drinks), plus=True)
      else:
        val["sodiumletter"] = per_value_letter(float(val[k]), "sodium", drink=(val["gtin"] in drinks), plus=True)
    if "protein" in k and not "proteinletter" in k:
      if k in nutris[val["gtin"]]:
        val["proteinletter"] = per_value_letter(float(nutris[val["gtin"]][k]), "protein", drink=(val["gtin"] in drinks), plus=False)
      else:
        val["proteinletter"] = per_value_letter(float(val[k]), "protein", drink=(val["gtin"] in drinks), plus=False)
    if "fiber" in k and not "fiberletter" in k:
      if k in nutris[val["gtin"]]:
        val["fiberletter"] = per_value_letter(float(nutris[val["gtin"]][k]), "fiber", drink=(val["gtin"] in drinks), plus=False)
      else:
        val["fiberletter"] = per_value_letter(float(val[k]), "fiber", drink=(val["gtin"] in drinks), plus=False)
    for k in ks:
      if k == "id":
        continue
      elif k == "name":
        print('\t\t"{}":"{}",'.format(k,val[k]))
      elif k in nutris[val["gtin"]]:
        print('\t\t"{}":"{}",'.format(k, nutris[val["gtin"]][k]))
      else:
        print('\t\t"{}":"{}",'.format(k,val[k]))
    print("\t},")
  
  if val["gtin"] not in nutris:
    if not val["gtin"] == "":
      print('\t"{}":'.format(val["gtin"]))
      print("\t{")
      for k,v in val.items():
        if not k == "id":
          print('\t\t"{}":"{}",'.format(k,v))
      print("\t},")

extranutri = {}
for key, val in category_index.items():
  if val["gtin"] not in nutris and val["gtin"] == "":
    for k, v in extranutri.items():
      if val["name"].split("_")[0:5] == v["name"].split("_")[0:5]:
        val = v
        key = k
        if "energy" in val:
            val["energyletter"] = per_value_letter(float(val["energy"]), "energy", drink=(val["gtin"] in drinks), plus=True)
        if "sugar" in val:
            val["sugarletter"] = per_value_letter(float(val["sugar"]), "sugar", drink=(val["gtin"] in drinks), plus=True)
        if "sat_fat" in val:
            val["sat_fatletter"] = per_value_letter(float(val["sat_fat"]), "sat_fat", drink=(val["gtin"] in drinks), plus=True)
        if "sodium" in val:
            val["sodiumletter"] = per_value_letter(float(val["sodium"]), "sodium", drink=(val["gtin"] in drinks), plus=True)
        if "protein" in val:
            val["proteinletter"] = per_value_letter(float(val["protein"]), "protein", drink=(val["gtin"] in drinks), plus=False)
        if "fibre" in val:
            val["fibreletter"] = per_value_letter(float(val["fiber"]), "fiber", drink=(val["gtin"] in drinks), plus=False)
            val["fiberletter"] = per_value_letter(float(val["fiber"]), "fiber", drink=(val["gtin"] in drinks), plus=False)
        break
    print('\t"{}":'.format(val["gtin"]))
    print("\t{")
    for k,v in val.items():
      if not k == "id":
        print('\t\t"{}":"{}",'.format(k,v))
    print("\t},")

In [0]:
for id, val in category_index.items():
  if val["gtin"] == "":
    continue
  print('\t\t\tcase {}:'.format(id))
  print('\t\t\t\tproductDefinition.id = "{}";'.format(id))
  print('\t\t\t\tproductDefinition.name = "{}";'.format(nutris[val["gtin"]]["name"]))
  print('\t\t\t\tproductDefinition.gtin = "{}";'.format(nutris[val["gtin"]]["gtin"]))
  print('\t\t\t\tproductDefinition.nutri_score = {};'.format(nutris[val["gtin"]]["nutriscore"]))
  print('\t\t\t\tproductDefinition.nutri_label = "{}";'.format(nutris[val["gtin"]]["nutriletter"]))
  print('\t\t\t\tproductDefinition.calories = "{}";'.format(nutris[val["gtin"]]["energy"]))
  print('\t\t\t\tproductDefinition.caloriesColor = "{}";'.format(nutris[val["gtin"]]["energyletter"]))
  if "sugar" in nutris[val["gtin"]]:
    print('\t\t\t\tproductDefinition.sugar = "{}";'.format(nutris[val["gtin"]]["sugar"]))
    print('\t\t\t\tproductDefinition.sugarColor = "{}";'.format(nutris[val["gtin"]]["sugarletter"]))
  else:
    print('\t\t\t\tproductDefinition.sugar = "0";')
    print('\t\t\t\tproductDefinition.sugarColor = "B";')
  if "sat_fat" in nutris[val["gtin"]]:
    print('\t\t\t\tproductDefinition.fat = "{}";'.format(nutris[val["gtin"]]["sat_fat"]))
    print('\t\t\t\tproductDefinition.fatColor = "{}";'.format(nutris[val["gtin"]]["sat_fatletter"]))
  else:
    print('\t\t\t\tproductDefinition.fat = "0";')
    print('\t\t\t\tproductDefinition.fatColor = "B";')
  if "protein" in nutris[val["gtin"]]:
    print('\t\t\t\tproductDefinition.protein = "{}";'.format(nutris[val["gtin"]]["protein"]))
    print('\t\t\t\tproductDefinition.proteinColor = "{}";'.format(nutris[val["gtin"]]["proteinletter"]))
  else:
    print('\t\t\t\tproductDefinition.protein = "0";')
    print('\t\t\t\tproductDefinition.proteinColor = "C";')
  if "fiber" in nutris[val["gtin"]]:
    print('\t\t\t\tproductDefinition.fiber = "{}";'.format(nutris[val["gtin"]]["fiber"]))
    print('\t\t\t\tproductDefinition.fiberColor = "{}";'.format(nutris[val["gtin"]]["fiberletter"]))
  else:
    print('\t\t\t\tproductDefinition.fiber = "0";')
    print('\t\t\t\tproductDefinition.fiberColor = "C";')
  print('\t\t\t\tbreak;')

In [0]:
cnt = 1
for id, val in nutris_new.items():
  pd_txt = 'productDefinition{}'.format(cnt)
  cnt += 1
  print('\t\tProductDefinition ' + pd_txt + ' = new ProductDefinition();')
  print('\t\t' + pd_txt + '.id = "{}";'.format(val["name"]))
  print('\t\t' + pd_txt + '.name = "{}";'.format(val["display_name"]))
  print('\t\t' + pd_txt + '.gtin = "{}";'.format(val["gtin"]))
  if val["isdrink"] or val["iswater"]:
    print('\t\t' + pd_txt + '.type = "Drink";')
  else:
    print('\t\t' + pd_txt + '.type = "Food";')
  print('\t\t' + pd_txt + '.nutri_score = {};'.format(val["nutriscore"]))
  print('\t\t' + pd_txt + '.nutri_label = "{}";'.format(val["nutriletter"]))
  print('\t\t' + pd_txt + '.calories = "{}";'.format(val["energy"]))
  print('\t\t' + pd_txt + '.caloriesColor = "{}";'.format(val["energyletter"]))
  print('\t\t' + pd_txt + '.sugar = "{}";'.format(val["sugar"]))
  print('\t\t' + pd_txt + '.sugarColor = "{}";'.format(val["sugarletter"]))
  print('\t\t' + pd_txt + '.fat = "{}";'.format(val["sat_fat"]))
  print('\t\t' + pd_txt + '.fatColor = "{}";'.format(val["sat_fatletter"]))
  print('\t\t' + pd_txt + '.protein = "{}";'.format(val["protein"]))
  print('\t\t' + pd_txt + '.proteinColor = "{}";'.format(val["proteinletter"]))
  print('\t\t' + pd_txt + '.fiber = "{}";'.format(val["fiber"]))
  print('\t\t' + pd_txt + '.fiberColor = "{}";'.format(val["fiberletter"]))
  print('\t\t' + pd_txt + '.natrium = "{:.2f}";'.format(val["natrium"]))
  print('\t\t' + pd_txt + '.natriumColor = "{}";'.format(val["natriumletter"]))
  print('\t\t' + pd_txt + '.healthpercentage = "{}";'.format(val["veggieratio"] * 100))
  print('\t\t' + pd_txt + '.healthpercentageColor = "{}";'.format(val["veggieratioletter"]))
  print('\t\tProductNutris.Add(' + pd_txt + '.id, ' + pd_txt + ');')
  print("")

In [0]:
print("nutris = {")
for id, val in nutris_new.items():
  print('\t"{}":'.format(val["name"]) + '{')
  print('\t\t"energy":"{}",'.format(val["energy"]))
  print('\t\t"sugar":"{}",'.format(val["sugar"]))
  print('\t\t"sat_fat":"{}",'.format(val["sat_fat"]))
  print('\t\t"natrium":"{}",'.format(val["natrium"]))
  print('\t\t"protein":"{}",'.format(val["protein"]))
  print('\t\t"fiber":"{}",'.format(val["fiber"]))
  print('\t\t"health_percentage":"{}",'.format(val["veggieratio"]))
  print('\t},')
print("}")


In [0]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

nutri_by_score = {it:{} for it in range(-15,41)}
for id, val in nutris_new.items():
  nutri_by_score[val["nutriscore"]][val["name"]] = ""
pp.pprint(nutri_by_score)

{ -15: { 'evian___50__3068320353500': '',
         'henniez_blau__33__7610235000442': '',
         'valser_classic__50__76404160': '',
         'valser_still__50__7610335002575': ''},
  -14: {},
  -13: {},
  -12: {},
  -11: {},
  -10: {},
  -9: {},
  -8: {},
  -7: {},
  -6: {},
  -5: {},
  -4: {},
  -3: {'powerbar_proteinplusschoko__55__4029679520028': ''},
  -2: {},
  -1: {},
  0: { 'coke_zero_flasche_50__5449000131836': '',
       'comella_schokodrink__33__7613100037253': '',
       'pepsi_max__50__4060800105943': ''},
  1: { 'airwaves_menthoneucalyptus_riegel_14_1_50173167': '',
       'redbull_light__33__90162800': '',
       'stimorol_spearmint_riegel_14_1_0000000000005': ''},
  2: { 'caferoyal_classicmacchiato__33__7617014164487': '',
       'stimorol_wildcherry_riegel_14_1_57060330': ''},
  3: {'balisto_muesli__37g__5000159418546': ''},
  4: { 'fuse_lemon_dose_33__5449000235947': '',
       'fuse_peach__50__5449000236623': '',
       'granini_orange__50__4002160092334': '',
    

## Request

In [0]:
def show_video_frame(image):
  """ Takes a video and a frame id as numeric
  and displays that frame inline
  Args:
    filepath: path to video file
    idx: id of frame as counted from start with 0
  """
  plt.figure()
  plt.imshow(image)
  plt.axis('off')
  plt.show()

In [0]:
def draw_per_frame(image, boxes, classes, scores, category_index):
  """for each image draw the boxes and the labels, with the color according
  to the nutriscore
  
  """
  boxes = np.reshape(boxes,[int(len(scores)),4])
  classes = np.squeeze(classes).astype(np.int32)
  scores = np.squeeze(scores)
  min_score_thresh = 0.8
  use_normalized_coordinates=True
  line_thickness=12
  nutri_colors = {"A": "Green",
                  "B": "LimeGreen",
                  "C": "Yellow",
                  "D": "Orange",
                  "E": "Red"}
  box_to_display_str_map = collections.defaultdict(list)
  box_to_color_map = collections.defaultdict(str)
  max_boxes_to_draw = boxes.shape[0]
  for i in range(min(max_boxes_to_draw, boxes.shape[0])):
    if scores is None or scores[i] > min_score_thresh:
      box = tuple(boxes[i].tolist())
      class_dict = category_index[classes[i]]
      if not "gtin" in class_dict:
        display_str = '{}: {}'.format(class_dict["id"], class_dict["name"])
        box_to_color_map[box] = "DarkGrey"
        print(class_dict)
      elif not "nutriscore" in class_dict:
        display_str = class_dict["gtin"]
        box_to_color_map[box] = "DarkGrey"
        print(class_dict)
      else:
        display_str = '{}: {}'.format(class_dict["name"], class_dict["nutriscore"])
        box_to_color_map[box] = nutri_colors[class_dict["nutriletter"]]
      box_to_display_str_map[box].append(display_str)
        
  # Draw all boxes onto image.
  for box, color in box_to_color_map.items():
    ymin, xmin, ymax, xmax = box
    visualization_utils.draw_bounding_box_on_image_array(
        image,
        ymin,
        xmin,
        ymax,
        xmax,
        color=color,
        thickness=line_thickness,
        display_str_list=box_to_display_str_map[box],
        use_normalized_coordinates=use_normalized_coordinates)

  return image

In [0]:
def create_video_from_frames(video_frames):
  """take the frames of the video and create
  a video from them.
  
  """
  height, width, layers = video_frames[0]["frame"].shape

  video = cv2.VideoWriter("video.mp4", cv2.VideoWriter_fourcc(*'MP4V'), 24, (width,height))

  for image in video_frames:
      image_bgr = cv2.cvtColor(image["frame"], cv2.COLOR_BGR2RGB)
      video.write(image_bgr)

  cv2.destroyAllWindows()
  video.release()

In [0]:
def resize_padded_with_bndboxes(width_old, height_old, filepath_old, bndboxs_old, length):
  """resizes the image ot a squared padded image. Where the longest
  side will match their target length. The other dimension is padded
  equally on both sides with zeros until it as well matches length.
  Args:
    width_old: the old width
    height_old: the old height
    filepath_old: the old filepath
    bndboxs: a list of bounding boxes
    length: target length
  Returns:
    width: the new width
    length: the new length
    filepath: the new filepath
    bndboxs: the new boundingboxes
  """
  #First resize the image
  width_old, height_old, filepath, bndboxs_old = resize_scaled_with_bndboxes(width_old,
                                                                 height_old,
                                                                 filepath_old,
                                                                 bndboxs_old,
                                                                 length)
  
  
  
  #create empty image array and embedd the image in it
  
  cv2.imwrite(filepath, image)
  
  #redo boundingboxes
  bndboxs = []
  for bndbox in bndboxs_old:
    bndboxs.append({})
    bndboxs[-1]["xmin"] = (bndbox["xmin"] * width_old + pad_y) / float(length)
    bndboxs[-1]["xmax"] = (bndbox["xmax"] * width_old + pad_y) / float(length)
    bndboxs[-1]["ymin"] = (bndbox["ymin"] * height_old  + pad_x) / float(length)
    bndboxs[-1]["ymax"] = (bndbox["ymax"] * height_old  + pad_x) / float(length)
    bndboxs[-1]["class_name"] = bndbox["class_name"]
    bndboxs[-1]["class"] = bndbox["class"]
    
    if not all([(float(val) >= 0.0 and float(val) <= 1.0) for key, val in bndboxs[-1].items() if key in ["xmin", "xmax", "ymin", "ymax"]]):
      tf.logging.info("bndboxes out of scope!!")
      tf.logging.info(bndboxs[-1])
      for key, val in bndboxs[-1].items():
        if key in ["xmin", "xmax", "ymin", "ymax"]:
          if val > 1.0:
            val = 1.0
          elif val < 0.0:
            val = 0.0
  
  return length, length, filepath, bndboxs

In [0]:
def resize_to_ratio(image, max_length = 640, padded=True):
  """resize the image to max_length along longest side
  and keep the ratio for the other side.
  Args:
    image: the image as numpy array
    max_length: max length in pixel along longest side
  Returns:
    the resized image
  """
  height_old, width_old, channels = image.shape
  if width_old > height_old:
    scaler = width_old / max_length
  else:
    scaler = height_old / max_length
  
  height = int(height_old / scaler) 
  width = int(width_old / scaler)
  
  image_new = cv2.resize(image, (width,height))

  if padded:
    #necessary padding, handle uneven numbers
    pad_x = 0
    pad_y = 0
    if height < max_length:
      pad_size = max_length - height
      pad_x = int(pad_size/2)
    if width < max_length:
      pad_size = max_length - width
      pad_y = int(pad_size/2)
    
    image_tmp = np.zeros([max_length, max_length, 3])
    print(pad_x, ":(",pad_x + height,"), ",pad_y,":(",pad_y + width,"),:")
    image_tmp[pad_x:(pad_x + height), pad_y:(pad_y + width), :] = image_new
    image_new = image_tmp
  
  return image_new
  

In [0]:
def setup_stub_gRPC(ip, port):
  channel = grpc.insecure_channel("{}:{}".format(ip,port))
  stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
  # Send request
  # See prediction_service.proto for gRPC request/response details.
  return stub

In [0]:
def predict_per_frame_gRPC(image, stub, image_form):
  request = predict_pb2.PredictRequest()
  request.model_spec.name = 'holoselecta'
  request.model_spec.signature_name = 'serving_default'
  tic = time()
  if image_form == 'image_tensor':
    image_data = image
    image_shape = [1] + list(image.shape)
  if image_form == 'encoded_image_string_tensor':
    image_data = cv2.imencode('.jpg', image)
    image_data = image_data[1].tostring()
    image_shape = [1]
  request.inputs['inputs'].CopyFrom(make_tensor_proto(image_data, shape=image_shape))
  result = stub.Predict(request, 20.0)  # 20 secs timeout
  toc = time()
  print("--> query time of {}s".format(toc-tic))
  return result

In [0]:
def predict_per_frame_REST(image, ip, port, image_form):
  # either { "b64": base64.b64encode(image)}
  #encoded_input_string = base64.b64encode(image.tobytes())
  #input_string = encoded_input_string.decode("utf-8")
  #data = json.dumps({"inputs":{'inputs':{ "b64": input_string}}})
  # or image.tolist()
  tic = time()
  if image_form == 'image_tensor':
    image_data = [image.tolist()]
    data = json.dumps({"inputs":{'inputs': image_data}})
  if image_form == 'encoded_image_string_tensor':
    image_data = cv2.imencode('.jpg', image)
    image_data = image_data[1].tostring()
    image_data = { "b64": base64.b64encode(image_data).decode('utf8')}
    data = json.dumps({"inputs": [image_data]})
  headers = {"content-type": "application/json"}
  toc = time()
  print("--> json encode time of {}s".format(toc-tic))
  tic = time()
  json_response = requests.post('http://{}:{}/v1/models/holoselecta:predict'.format(ip, port), data=data, headers=headers)
  toc = time()
  print("--> query time of {}s".format(toc-tic))
  tic = time()
  results = json.loads(json_response.text)
  toc = time()
  print("--> json decode time of {}s".format(toc-tic))
  return results

In [0]:
#ip = "34.76.251.244"
ip = "34.90.247.213"
port_rest = "80"
port_grpc = "443"
use_grpc = True
image_form = 'encoded_image_string_tensor'

image = cv2.imread("gdrive/My Drive/Masterarbeit/ObjectDetection/Images/_Automat/IMG_20190430_111555.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_resized = resize_to_ratio(image, 320, True)

if use_grpc:
  stub = setup_stub_gRPC(ip, port_grpc)
  results = predict_per_frame_gRPC(image_resized, stub, image_form)
  boxes = results.outputs['detection_boxes'].float_val
  classes = results.outputs['detection_classes'].float_val
  scores = results.outputs['detection_scores'].float_val
else:
  results = predict_per_frame_REST(image_resized, ip, port_rest, image_form)
  boxes = results['outputs']['detection_boxes'][0]
  classes = results['outputs']['detection_classes'][0]
  scores = results['outputs']['detection_scores'][0]

print(results)
  
image = draw_per_frame(image, boxes, classes, scores, category_index)
show_video_frame(image)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cv2.imwrite("gdrive/My Drive/Masterarbeit/bild1.png", image)

0 :( 320 ),  40 :( 280 ),:


_Rendezvous: ignored

In [0]:
image = cv2.imread("gdrive/My Drive/Masterarbeit/ObjectDetection/Images/poster/_Automat/DSC01632_775x14001.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_resized = resize_to_ratio(image)
print(len(str(image.tolist())))
encoded_input_string = base64.b64encode(image)
input_string = encoded_input_string.decode("utf-8")
print(len(input_string))

1036760128
277804800


In [0]:
#TODO OpenCV uses BGR instead of RGB adapt accordingly
#cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
video = "gdrive/My Drive/Masterarbeit/ObjectDetection/Video/holo_lens/20190111_133418_HoloLens.mp4"
vidcap = cv2.VideoCapture(video)
success,image = vidcap.read()
count = 0
ip = "34.90.247.213"
port_rest = "80"
port_grpc = "443"
use_grpc = True
image_form = 'encoded_image_string_tensor'
if use_grpc:
  stub = setup_stub_gRPC(ip, port_grpc)

video_frames = []
while success:
  try:
    print("finished frame {}".format(count))
    success, image = vidcap.read()
    count += 1
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = resize_to_ratio(image, 640, true)
    
    if use_grpc:
      results = predict_per_frame_gRPC(image_resized, stub, image_form)
      boxes = results.outputs['detection_boxes'].float_val
      classes = results.outputs['detection_classes'].float_val
      scores = results.outputs['detection_scores'].float_val
    else:
      results = predict_per_frame_REST(image_resized, ip, port_rest, image_form)
      boxes = results['outputs']['detection_boxes'][0]
      classes = results['outputs']['detection_classes'][0]
      scores = results['outputs']['detection_scores'][0]
    
    image_reprinted = draw_per_frame(image, boxes, classes, scores, category_index)
    video_frames.append({"frame": image_reprinted})
    
  except:
    print("--> failed")
    
  if count == 30:
    break

create_video_from_frames(video_frames)

finished frame 0
--> failed
finished frame 1
--> failed
finished frame 2
--> failed
finished frame 3
--> failed
finished frame 4
--> failed
finished frame 5
--> failed
finished frame 6
--> failed
finished frame 7
--> failed
finished frame 8
--> failed
finished frame 9
--> failed
finished frame 10
--> failed
finished frame 11
--> failed
finished frame 12
--> failed
finished frame 13
--> failed
finished frame 14
--> failed
finished frame 15
--> failed
finished frame 16
--> failed
finished frame 17
--> failed
finished frame 18
--> failed
finished frame 19
--> failed
finished frame 20
--> failed
finished frame 21
--> failed
finished frame 22
--> failed
finished frame 23
--> failed
finished frame 24
--> failed
finished frame 25
--> failed
finished frame 26
--> failed
finished frame 27
--> failed
finished frame 28
--> failed
finished frame 29
--> failed


IndexError: ignored

In [0]:
create_video_from_frames(video_frames)

# Stuff

These resolutions are problematic because we keep an entire queue of images in memory, not just the batch that you are currently training on. See e.g. the queue_capacity and min_after_dequeue parameters in https://github.com/tensorflow/models/blob/master/object_detection/protos/input_reader.proto

## Serving Setup GCE GPU

Launch latest deep learning tf gpu vm on GCE with nvidia drivers installed

gsutil cp gs://holoselecta_objectdetection/holoselecta.tar.gz holoselecta.tar.gz

docker pull docker.io/tensorflow/serving:latest-gpu

tar xvzf holoselecta.tar.gz  
rm holoselecta.tar.gz  
rm export/*/holoselecta.tar.gz  

docker run --name holoselecta --runtime=nvidia -p 443:8500 -p 80:8501 --mount type=bind,source=/home/grundtob/export/,target=/models/holoselecta -e MODEL_NAME=holoselecta -t tensorflow/serving:latest-gpu &

for logging:  -e TF_CPP_MIN_VLOG_LEVEL=1

## Setting Up GCE CPU
* gsutil cp gs://holoselecta_objectdetection/holoselecta.tar.gz holoselecta.tar.gz

* sudo docker pull docker.io/tensorflow/serving:latest

* tar xvzf holoselecta.tar.gz
* rm holoselecta.tar.gz
* rm export/*/holoselecta.tar.gz

* sudo docker run --name holoselecta -p 443:8500 -p 80:8501 --mount type=bind,source=/home/grundtob/export/,target=/models/holoselecta -e MODEL_NAME=holoselecta  -t tensorflow/serving:latest &

for logging: -e TF_CPP_MIN_VLOG_LEVEL=1

In [0]:
#TODO test inference on serving docker with GPU
#TODO configure model_server to handel batching and send a batch of images


## Optimizations Serving
* compile improved binaries : https://www.tensorflow.org/serving/docker#building_an_optimized_serving_binary 

* follow issue: https://github.com/tensorflow/models/issues/3270

* follow advice: https://stackoverflow.com/questions/47896671/tensorflow-object-detection-api-has-slow-inference-time-with-tensorflow-serving/

* test if transfering the image as bytes directly is better than transfering it as a np.array
  